# Goal: creating a mini-site using Streamlit/Dash, where you can write down your ticker and all the data will be mined automatically
+ add y/y growth to every cash-flow/balance/income
+ add forecasts to plots (ARIMA or smth like that) + forecasts from analytics (seekingalpha.com) + Keras LSTM
+ add portfolio analysis (at least from etoro): sectors + dividend and price forecasting
+ add explanation to different ratios and indicators
+ add ETF analytics - sectors, top holdings (stockanalysis.com), overall position of different firms in portfolio
+ add different stuff based on alphaspread and seekingalpha examples
+ add news aggregator (maybe play a little bit with NLTK for text recognition and classification)
+ add grades from analytics and firms (current and historical, checked using logistic regression??)
+ add Monte Carlo simulation based on historical performance
+ add somewhere ML / Gradient Boosting / Decision Tree / etc. ??
+ add this to Streamlit (Heroku?), in order to do it -> .py. not .ipynb
+ add all of this to AWS (or other Cloud) for data to be updated automatically?

In [1]:
# finance api
import yahoofinancials
import yahooquery as yq
import yfinance as yf
import nasdaqdatalink as ndl

# data
import numpy as np
import pandas as pd
import datetime as dt

# visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
import streamlit as st
from plotly.subplots import make_subplots

# web scrapping
import requests
import json
import time
import re
import lxml
import cchardet
from bs4 import BeautifulSoup

NASDAQ_DATA_LINK_API_KEY = "xy8jtvPFDhiwnFktEugz"  # ndl.ApiConfig.api_key
pd.set_option("display.max_columns", None)

# Set the template to 'plotly_dark'
pio.templates.default = "plotly_dark"


In [2]:
STOCK = "MSFT"

STOCKS_LIST = [
    "AAPL",
    "ABBV",
    "ABR",
    "ABT",
    "AFL",
    "AMD",
    "AMGN",
    "AMZN",
    "ASML",
    "ATVI",
    "AVGO",
    "BBY",
    "BHP",
    "BRK-B",
    "CCI",
    "COST",
    "CSCO",
    "CVX",
    "GOOGL",
    "HD",
    "IRM",
    "JNJ",
    "JPM",
    "LOW",
    "MA",
    "MAIN",
    "MCD",
    "MCO",
    "MDT",
    "META",
    "MRK",
    "MS",
    "MSFT",
    "NFLX",
    "NVDA",
    "O",
    "PEP",
    "PFE",
    "PLD",
    "PYPL",
    "SBUX",
    "STOR",
    "TGT",
    "TROW",
    "TSLA",
    "TSM",
    "UNP",
    "UPS",
    "V",
    "VICI",
    "WPC",
    "XOM",
    "ZTS",
]

CRYPTO_LIST = ["BTC-USD", "ETH-USD"]

ETF_LIST = ["SCHD", "SPHD", "VOO", "QQQ", "VGT", "ARKK"]

WATCHLIST_LIST = ["MO", "T"]


In [3]:
stock_list_new = [x.lower() if x != 'BRK-B' else 'brk.b' for x in STOCKS_LIST]
stock_list_test = [STOCK]

In [4]:
def get_macrotrends_data(url: str):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    scripts = soup.find_all("script", type="text/javascript")

    for script in scripts:
        if "originalData" in script.text:
            var_data = script.text

    # Use regular expressions to extract the variable
    match = re.search("var originalData = (.*);", var_data)
    data = match.group(1)
    df = pd.read_json(data)

    for i, k in enumerate(df["field_name"]):
        if BeautifulSoup(k, "html.parser").find("a"):
            df.loc[i, "field_name"] = BeautifulSoup(k, "html.parser").find("a").text
        else:
            df.loc[i, "field_name"] = BeautifulSoup(k, "html.parser").find("span").text

    df = (
        df.drop(columns=["popup_icon"])
        .set_index("field_name")
        .replace("", 0)
        .astype(float)
        .T
    )
    return df


In [5]:
def create_plot_bar_line(
    df: pd.DataFrame,
    bar="",
    line="",
    y2perc=False,
    secondary_y=True,
    bar_color="#5a7d9f",
    line_color="white",
    title="",
):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_bar(
        x=df.index, y=df[bar], name=bar, secondary_y=False, marker_color=bar_color
    )

    if title != "":
        title_ = title
    else:
        title_ = bar

    fig.update_layout(
        height=400,
        width=600,
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=title_,
        legend=dict(
            orientation="h", yanchor="bottom", y=-0.15, xanchor="right", x=0.75
        ),
    )

    if line != "":
        fig.add_scatter(
            x=df.index,
            y=df[line],
            name=line,
            secondary_y=secondary_y,
            marker_color=line_color,
        )
        if y2perc:
            fig.update_layout(yaxis2=dict(tickformat=".0%"))

    fig.show()

    return fig


In [6]:
def create_ema_plot(tickers_list, start_date="2019-01-01", period="days", emas=[1]):

    colors = {
        "standard": "#424242",
        "min": "#00ab41",
        "other": "#8B8000",
        "max": "#c30010",
    }

    df_orig = pd.DataFrame(yf.download(tickers_list, start_date)["Adj Close"])

    if len(tickers_list) > 1:
        vertical_spacing = (1 / (len(tickers_list) - 1)) / 9
    else:
        vertical_spacing = 0
        df_orig.columns = tickers_list

    df = df_orig.copy()

    for n in emas:
        ema_period = f"{n} {period}"
        ema_df = df_orig.ewm(
            halflife=ema_period, times=df_orig.index, ignore_na=False
        ).mean()
        ema_df.columns = [f"{c}_{n}{period}_ema" for c in ema_df.columns]
        df = pd.concat([df, ema_df], axis=1)

    fig = make_subplots(
        rows=len(tickers_list),
        cols=1,
        subplot_titles=tickers_list,
        vertical_spacing=vertical_spacing,
    )

    for i, c in enumerate(tickers_list, start=1):
        df_columns = [col for col in df.columns if col.startswith(c + "_") | (col == c)]
        df_new = df[df_columns].copy()
        for i2, c2 in enumerate(df_columns):

            if i2 == 0:
                color = colors["standard"]
                width = 2
            elif i2 == 1:
                color = colors["min"]
                width = 2
            elif i2 == len(df_columns) - 1:
                color = colors["max"]
                width = 2
            else:
                color = colors["other"]
                width = 0.5

            fig.add_trace(
                go.Scatter(
                    x=df_new.index,
                    y=df_new[c2],
                    line=dict(color=color, width=width),
                    name=c2,
                ),
                row=i,
                col=1,
            )

    fig.update_layout(
        height=len(tickers_list) * 400,
        width=600,
        showlegend=False,
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
    )

    fig.show()

    return df


In [7]:
def create_line_plot(df: pd.DataFrame, y: list, title="", perc=True):
    fig = go.Figure()

    for line in y:
        fig.add_scatter(x=df.index, y=df[line], name=line)

    fig.update_layout(
        height=400,
        width=600,
        legend=dict(
            orientation="h", yanchor="bottom", y=-0.25, xanchor="right", x=0.75
        ),
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=title,
    )

    if perc:
        fig.update_layout(yaxis=dict(tickformat=".1%"))

    fig.show()
    return fig


# API

Dividend data

In [8]:
def create_div_history_df(stock_list = stock_list_test):
    """Seeking alpha full dividend history"""
    div_history_df = pd.DataFrame()

    for tick in stock_list:
        url = f"https://seekingalpha.com/api/v3/symbols/{tick.lower()}/dividend_history"
        querystring = {"years":"100"}
        headers = {
            "cookie": "machine_cookie=4979826528810; _cls_v=072cd8fc-83ec-4b6d-b840-72ce92a351d4; _cls_s=da78f999-6e82-4412-bfd3-98a35379d96d:0; _pxvid=6190f403-0540-11ed-8356-71796f6e5767; pxcts=61910480-0540-11ed-8356-71796f6e5767; g_state=^{^\^i_l^^:0^}; has_paid_subscription=false; OptanonAlertBoxClosed=2022-07-16T19:49:37.138Z; _ga=GA1.2.422884809.1658000977; _igt=80f0662b-29d6-4ba2-daef-f15a084be986; _hjSessionUser_65666=eyJpZCI6IjVmNjA3NTU1LTFmODItNWFhOC05NzBkLTMxNmIwOTFkNDJjZSIsImNyZWF0ZWQiOjE2NTgwNDMwMjQxNTYsImV4aXN0aW5nIjp0cnVlfQ==; _hjCachedUserAttributes=eyJhdHRyaWJ1dGVzIjp7ImxvZ2dlZF9pbiI6dHJ1ZSwibXBfc3ViIjpmYWxzZSwicHJlbWl1bV9zdWIiOmZhbHNlLCJwcm9fc3ViIjpmYWxzZX0sInVzZXJJZCI6IjU2ODczOTA0In0=; ga_clientid=422884809.1658000977; _pcid=^%^7B^%^22browserId^%^22^%^3A^%^22l6l1zvh16ggo2rl5^%^22^%^7D; _clck=1sv21qj^|1^|f4c^|0; _ig=56873904; sailthru_content=2528dc295dc3fbbf1ec8e71fd6af16ea5ed0fab1751712d30b586234ac21ac69c6f48017810681510ac670347a1b237b395addcc8a084ec17e397065464a467803e85c27969d6ca11adf1e5bae9ce43e365ade53ba1716e0f5409199ca81b1b2d336ff2bdab2770099e746360c3b2e4a8f46c8cbd3b263891ad28c66986af90e8a2bb0fb3446957f12521164830063aa9eada221935b05aaed9d45ccc5957509; sailthru_visitor=4a85db3b-194e-42bd-bc87-31076f836304; sailthru_hid=29f91ce2c0119534955a4934eea65d5d62d3164919e4cd8e5507453023d2712d74fca4d95585b51117583622; _gcl_au=1.1.905016176.1671643238; __pat=-18000000; user_id=56873904; user_nick=; user_devices=2; u_voc=; marketplace_author_slugs=; user_cookie_key=cjjdiz; user_perm=; sapu=101; user_remember_token=04b7dcb2602e3f78db1c7c7b3e0e43599aa202f5; _sapi_session_id=0pCP6BL7ckaTjzz1yGfnvj2fYymMCVyRcdc0FilJJuJrLs^%^2BPk6M7pmkTNZq^%^2Bs0tQzLw0Gwxfpuz4XXdeLwjnEvGdwVGKVQdIhiI4kf6GgA6c6Aqo8EAHDVX3JUirUkOfv7^%^2Fv6zuUolHyz^%^2Bka3l7tx2Tmr6LfeaHe0syKkJJ99iSM^%^2FbcPrEEdST3wciFuUBwzxt3V9trL98gAlWdoY4Ces0hsdCU^%^2BEryApHpHc9rt8S2ZjmXsQ7PNxkHufEwIxhqC2LmTKsoVyrOgYz4rWUiq8CGM^%^2BdxILxHnEzl1LN9h2hU^%^3D--^%^2Fq^%^2FbqzYaui40jz7x--I^%^2FfbuLyN7DqYI^%^2BHocBaR9A^%^3D^%^3D; _pctx=^%^7Bu^%^7DN4IgrgzgpgThIC5QFYBsAOA7AZgJwAYAWRUABxigDMBLAD0RBABoQAXAT1KgYDUANEAF9BLSLADKrAIatIDCgHNqEVrCgATZiAjVVASU0IAdmAA2pwUA; _pxhd=9b81b7053d831d0e418b92698dce0fc88c8297e1e67eb88e98fefc26b9d3b6ac:80650f60-6b3b-11e9-814e-41aaaa844f02; ubvt=b26b3487-0e8c-451d-9656-705df157b6a2; session_id=27a89810-0094-4454-8793-f52f76340fbd; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+22+2022+16^%^3A05^%^3A26+GMT^%^2B0100+(czas+^%^C5^%^9Brodkowoeuropejski+standardowy)&version=6.30.0&landingPath=NotLandingPage&groups=C0001^%^3A1^%^2CC0002^%^3A1^%^2CC0003^%^3A1^%^2CC0007^%^3A1&hosts=H40^%^3A1^%^2CH17^%^3A1^%^2CH13^%^3A1^%^2CH36^%^3A1^%^2CH55^%^3A1^%^2CH69^%^3A1^%^2CH45^%^3A1^%^2CH14^%^3A1^%^2CH15^%^3A1^%^2CH19^%^3A1^%^2CH47^%^3A1&AwaitingReconsent=false&genVendors=V12^%^3A1^%^2CV5^%^3A1^%^2CV7^%^3A1^%^2CV8^%^3A1^%^2CV13^%^3A1^%^2CV15^%^3A1^%^2CV3^%^3A1^%^2CV2^%^3A1^%^2CV6^%^3A1^%^2CV14^%^3A1^%^2CV1^%^3A1^%^2CV4^%^3A1^%^2CV9^%^3A1^%^2C&geolocation=PL^%^3B14; __pnahc=1; gk_user_access=1**1671790151; gk_user_access_sign=316999477f1cf3b270ec2daee33355ef077c23cf; __tac=; __tae=1671790157992; LAST_VISITED_PAGE=^%^7B^%^22pathname^%^22^%^3A^%^22https^%^3A^%^2F^%^2Fseekingalpha.com^%^2Fsymbol^%^2FDPZ^%^2Fdividends^%^2Fhistory^%^22^%^2C^%^22pageKey^%^22^%^3A^%^22ba85820c-c9a7-4301-91ed-047be2dec0c2^%^22^%^7D; _uetsid=c9555410815311ed8383e1bd89176270; _uetvid=6c9a7a40054011ed9912e34a5318d584; __pvi=eyJpZCI6InYtMjAyMi0xMi0yMy0xMS0wOS0xNC0zMDYtRFVlQXM1NWtGcHdFelhldy05OWVlM2VhYmJkMDU0N2NiMjRiMjQ2ZTU5ZTc4YmQ4OCIsImRvbWFpbiI6Ii5zZWVraW5nYWxwaGEuY29tIiwidGltZSI6MTY3MTc5MDY2ODc4NH0^%^3D; __tbc=^%^7Bkpex^%^7Dc34b4dUSkelinBilgVjlXAFjdExL2yDTVVsaH2tHeWieSgu52a503DdkAZX5En4R; xbc=^%^7Bkpex^%^7DpsZvcg-czvsWNhuvqvMZK8J5UpYhUPaAf31G9LNO4s_JNybiiLibHlVRHn3hm4E4nn-OgFei0KNGMmPkAUA1_w-h83kuroSVs6Wm4u7Ywo2khMWDgt1X4fFsw_eRSpv_RT073ml6wbguc-BKt5xBC3jze6MTqMhOTtHPaQlo8jgrWISTUeJdpSW5wg1k8whSzoS5_JJNFGD12hP_7LIJ9Rcboio5C_pfp4SlYIgOvl0t0F4JUlwH3AItmjnB36P2lQd46Wi4gj8SrJp-WVo44vskLuAbTmezh-9Nmb6v2dAtnefy1d_SnhK1ucoCCPyx9eHnXkzHTxLTKoa4V1CaJBGXBFnLuyNvM48L074T6SRARQTZyVNljtYreNy7Uxb-agK4V0R54vP3iIc0NEPleFizxGh8FZZoF4flQb7mGezf-1HBFpWUlIR7p55GktmivP2SWPpXI1SzKXApvhhYN_mlYAm6eHG7Pq1LZgIR4zWUkv2RKy3rJd9Qsk8cHLPlvjhuRmx_t1ZjQa7IsxW7_03FS_lF67VC3PfVw_sI7vJlVj9ccU7hT9ptOtwx7ECKKYPkv5zP7q_a3Yubi4CmIM5MP-cJhy_-6RU96KhQ-FqXxVYETn_nJbtT3MXgwQma1soxbODUZ0d9NKNDWU5_lu9l2WXp88Vf-PdLt9LNv-Q",
            "authority": "seekingalpha.com",
            "referer": f"https://seekingalpha.com/symbol/{tick}/dividends/history",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
        }
        response = requests.request("GET", url, headers=headers, params=querystring).json()['data']

        for id in response:
            row = pd.DataFrame([id['attributes']])
            row['Ticker'] = tick
            div_history_df = pd.concat([div_history_df, row], axis=0)

        if len(stock_list_test) >1:
            time.sleep(2) # in case of generating data on more then 1 ticker

    for c in [c for c in div_history_df.columns if '_date' in c]:
        div_history_df[c] = pd.to_datetime(div_history_df[c])

    div_history_df['amount'] = div_history_df['amount'].astype(float)
    div_history_df = div_history_df.reset_index(drop=True).set_index('date')

    return div_history_df

In [9]:
div_history_df = create_div_history_df()

In [10]:
"""NASDAQ dividend data"""

url = f"https://api.nasdaq.com/api/quote/{STOCK}/dividends"
querystring = {"assetclass": "stocks"}

headers = {
    "authority": "api.nasdaq.com",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
}
response = requests.request("GET", url, headers=headers, params=querystring).json()

ex_div_date = response["data"]["exDividendDate"]
div_pay_date = response["data"]["dividendPaymentDate"]
div_yield = response["data"]["yield"]
annualized_div = response["data"]["annualizedDividend"]

# dividend_history = pd.DataFrame(response['data']['dividends']['rows'])

# for c in [c for c in dividend_history.columns if 'Date' in c]:
#     dividend_history[c] = pd.to_datetime(dividend_history[c])
# dividend_history = dividend_history.set_index('recordDate')
# dividend_history['amount'] = dividend_history['amount'].str.replace('$', '').astype(float)

# create_plot_bar_line(dividend_history, 'amount', title='NASDAQ dividend history')

print(f"ex_div_date: {ex_div_date}")
print(f"div_pay_date: {div_pay_date}")
print(f"div_yield: {div_yield}")
print(f"annualized_div: {annualized_div}")


ex_div_date: 11/16/2022
div_pay_date: 12/08/2022
div_yield: 1.1%
annualized_div: 2.72


StockAnalysis Financials

In [11]:
def create_income_statement(period = 'quarterly', stock_list = stock_list_test):
    period = 'quarterly'
    income_statement = pd.DataFrame()

    for x in stock_list: # STOCKS_LIST
        print(x)

        url = f"https://stockanalysis.com/stocks/{x.lower()}/financials/__data.json"

        querystring = {"x-sveltekit-invalidated":"__1"} # also period can be "trailing" or no period at all (annual)
        if (period == 'quarterly')|(period == 'trailing'):
            querystring['period'] = period

        headers = {
            "authority": "stockanalysis.com",
            "accept": "*/*",
            "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7,uk-UA;q=0.6,uk;q=0.5,pl;q=0.4",
            "cookie": "cf_chl_2=9c80dd02f1fc73c; cf_clearance=OwRTeLsjteKSq2vOGA415U77v0RksWzpM_0xtiixnIA-1671821196-0-160",
            "referer": f"https://stockanalysis.com/stocks/{x.lower()}/financials/",
            "sec-ch-ua": "^\^Not?A_Brand^^;v=^\^8^^, ^\^Chromium^^;v=^\^108^^, ^\^Google",
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "^\^Windows^^",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
        }
        
        response = requests.request("GET", url, headers=headers, params=querystring).json()

        # with open("stock_analysis.json", "w") as outfile:
        #     json.dump(response, outfile)

        response_new = response['nodes']
        for i in response_new:
            if i['type'] != 'skip':
                data = list(i['data'])
        
        dict1 = {}
        for k, v in zip(data[0].keys(), data[0].values()):
            dict1[k] = data[v]

        dict2 = {}
        for k, v in zip(dict1['financialData'].keys(), dict1['financialData'].values()):
            dict2[k] = data[v]

        dict3 = {}
        for k, v_list in zip(dict2.keys(), dict2.values()):
            values = []
            for v in v_list:
                values = values + [data[v]]
            dict3[k] = values

        dict_names = {}
        for m in dict1['map']:
            dict_names[data[data[m]['id']]] = data[data[m]['title']]
        dict_names['fcf'] = 'Free Cash Flow'
        dict_names['datekey'] = 'Date'

        income_df = pd.DataFrame(dict3)
        income_df['Ticker'] = x
        income_df = income_df.rename(columns=dict_names)
        
        income_statement = pd.concat([income_statement, income_df], axis=0)

        # time.sleep(4) # in case of generating data on more then 1 ticker

    income_statement = income_statement.set_index('Date')
    income_statement['Dividends'] = income_statement['Dividend Per Share'] * income_statement['Shares Outstanding (Basic)']

    return income_statement

In [12]:
income_statement = create_income_statement()
income_statement.head()

MSFT


,Revenue,Revenue Growth (YoY),Cost of Revenue,Gross Profit,"Selling, General & Admin",Research & Development,Operating Expenses,Other Operating Expenses,Operating Income,Interest Expense / Income,Other Expense / Income,Pretax Income,Income Tax,Preferred Dividends,Net Income,Net Income Growth,Shares Outstanding (Basic),Shares Outstanding (Diluted),Shares Change,EPS (Basic),EPS (Diluted),EPS Growth,Free Cash Flow Per Share,Dividend Per Share,Dividend Growth,Free Cash Flow,Gross Margin,Operating Margin,Profit Margin,Free Cash Flow Margin,Effective Tax Rate,EBITDA,EBITDA Margin,Depreciation & Amortization,EBIT,EBIT Margin,Ticker,Dividends
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-12-31,52747000000,0.019699,17488000000,35259000000,8016000000,6844000000,14860000000,0,20399000000,490000000,-430000000,20339000000,3914000000,0,16425000000,-0.124700,7451000000,7473000000,-0.010854,2.20,2.20,-0.112903,0.657,0.68,0.096774,4899000000,0.668455,0.386733,0.311392,0.092877,0.192438,24477000000,0.464045,3648000000,20829000000,0.394885,MSFT,5.066680e+09
2022-09-30,50122000000,0.106031,15452000000,34670000000,6524000000,6628000000,13152000000,0,21518000000,500000000,-554000000,21572000000,4016000000,0,17556000000,-0.143819,7457000000,7485000000,-0.010837,2.35,2.35,-0.132841,2.268,0.62,0.107143,16915000000,0.691712,0.429312,0.350265,0.337477,0.186167,24862000000,0.496030,2790000000,22072000000,0.440366,MSFT,4.623340e+09
2022-06-30,51865000000,0.123787,16429000000,35436000000,8053000000,6849000000,14902000000,0,20534000000,496000000,-449000000,20487000000,3747000000,0,16740000000,0.017135,7472000000,7504000000,-0.010157,2.24,2.24,0.032258,2.377,0.62,0.107143,17758000000,0.683235,0.395912,0.322761,0.342389,0.182896,24962000000,0.481288,3979000000,20983000000,0.404570,MSFT,4.632640e+09
2022-03-31,49360000000,0.183523,15615000000,33745000000,7075000000,6306000000,13381000000,0,20364000000,503000000,-329000000,20190000000,3462000000,0,16728000000,0.082228,7493000000,7534000000,-0.008293,2.23,2.22,0.093596,2.675,0.62,0.107143,20046000000,0.683651,0.412561,0.338898,0.406118,0.171471,24466000000,0.495665,3773000000,20693000000,0.419226,MSFT,4.645660e+09
2021-12-31,51728000000,0.200854,16960000000,34768000000,6763000000,5758000000,12521000000,0,22247000000,525000000,-793000000,22515000000,3750000000,0,18765000000,0.213542,7505000000,7555000000,-0.008009,2.50,2.48,0.221675,1.148,0.62,0.107143,8615000000,0.672131,0.430077,0.362763,0.166544,0.166556,26536000000,0.512991,3496000000,23040000000,0.445407,MSFT,4.653100e+09


In [13]:
def create_expenses_df(df=income_statement):
    # expenses dataframe for plot
    expenses = [
        'Selling & Marketing',
        'General & Administrative',
        'Selling, General & Admin',
        'Research & Development',
        'Other Operating Expenses',
        "Interest Expense",
        'Interest Expense / Income',
        'Other Expense / Income'
        ]
    df_expenses = df.loc[:,[c for c in df.columns if c in expenses]]
    for c in df_expenses.columns:
        if 'Expense / Income' in c:
            for i in df_expenses.index:
                if df_expenses.loc[i,c]>0:
                    df_expenses.loc[i,c] = 0
                else:
                    df_expenses.loc[i,c] = -df_expenses.loc[i,c]
    return df_expenses

In [14]:
df_expenses = create_expenses_df(df=income_statement)
df_expenses.head()

,"Selling, General & Admin",Research & Development,Other Operating Expenses,Interest Expense / Income,Other Expense / Income
Date,,,,,
2022-12-31,8016000000,6844000000,0,0,430000000
2022-09-30,6524000000,6628000000,0,0,554000000
2022-06-30,8053000000,6849000000,0,0,449000000
2022-03-31,7075000000,6306000000,0,0,329000000
2021-12-31,6763000000,5758000000,0,0,793000000


In [15]:
revenue_plot = create_plot_bar_line(
    income_statement, "Revenue", "Net Income", secondary_y=False # Cost of Revenue
)  
ebitda_plot = create_plot_bar_line(
    income_statement, "EBITDA", "EBITDA Margin", y2perc=True, bar_color="#805d67"
)
shares_plot = create_plot_bar_line(
    income_statement,
    "Shares Outstanding (Basic)",
    "Shares Change",
    y2perc=True,
    bar_color="#ea7726",
)
fcf_plot = create_plot_bar_line(
    income_statement, "Free Cash Flow", "Free Cash Flow Per Share", bar_color="#8d8b55"
)
profit_plot = create_plot_bar_line(
    income_statement,
    "Gross Profit",
    "Operating Expenses",
    secondary_y=False,
    bar_color="#7c459c",
)
dividends_plot = create_plot_bar_line(
    income_statement,
    "Dividend Per Share",
    "Dividend Growth",
    y2perc=True,
    bar_color="#03c03c",
)
margins_plot = create_line_plot(
    income_statement,
    y=["Gross Margin", "Operating Margin", "Profit Margin", "Free Cash Flow Margin"],
    title="Margins",
)
ema_plot = create_ema_plot([STOCK], emas=[10, 20, 30, 40, 50])

# eps_plot = create_plot_bar_line(income_statements, 'EPS (Basic)', 'EPS Growth', y2perc=True, bar_color='#7eb37a')
# dividends_full_history = create_plot_bar_line(div_history_df, 'amount', 'adjusted_amount', title='Full dividend history', bar_color='#03c03c')

[*********************100%***********************]  1 of 1 completed


In [16]:
# crypto_df = create_ema_plot(CRYPTO_LIST, emas=[10, 20, 30, 40, 50])
# stocks_df = create_ema_plot(STOCKS_LIST, emas=[10, 20, 30, 40, 50])

In [17]:
def create_stacker_bar(
    df: pd.DataFrame, title_="", colors=px.colors.sequential.Viridis
):

    stacked_plot = px.bar(data_frame=df, color_discrete_sequence=colors)

    stacked_plot.update_layout(
        xaxis=dict(title=None),
        yaxis=dict(title=None),
        height=400,
        width=600,
        legend=dict(
            orientation="h", yanchor="bottom", y=-0.4, xanchor="left", title=None
        ),
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        title=title_,
    )
    stacked_plot.show()

    return stacked_plot


In [18]:
expenses_plot = create_stacker_bar(df_expenses, title_='Expenses', colors=px.colors.sequential.Turbo_r)

macrotrends

In [19]:
tickers_macrotrends_dict = {}
macrotrends_list = requests.get(
    "https://www.macrotrends.net/assets/php/ticker_search_list.php?_=1673472383864"
).json()

for e in macrotrends_list:
    url_link = list(e.values())[1]
    ticker = list(e.values())[0].split(" - ")[0]
    tickers_macrotrends_dict[ticker] = url_link
# tickers_macrotrends_dict


In [20]:
# balance
balance = get_macrotrends_data(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/balance-sheet?freq=Q"
)  # ?freq=A
balance.head()


field_name,Cash On Hand,Receivables,Inventory,Pre-Paid Expenses,Other Current Assets,Total Current Assets,"Property, Plant, And Equipment",Long-Term Investments,Goodwill And Intangible Assets,Other Long-Term Assets,Total Long-Term Assets,Total Assets,Total Current Liabilities,Long Term Debt,Other Non-Current Liabilities,Total Long Term Liabilities,Total Liabilities,Common Stock Net,Retained Earnings (Accumulated Deficit),Comprehensive Income,Other Share Holders Equity,Share Holder Equity,Total Liabilities And Share Holders Equity
2022-12-31,99508.0,35833.0,2980.0,0.0,19502.0,157823.0,82755.0,7097.0,78259.0,24994.0,206729.0,364552.0,81718.0,44119.0,40648.0,99698.0,181416.0,90225.0,99368.0,-6457.0,0.0,183136.0,364552.0
2022-09-30,107262.0,31279.0,4268.0,0.0,18003.0,160812.0,77037.0,6839.0,78267.0,23482.0,198972.0,359784.0,87389.0,45374.0,39023.0,98829.0,186218.0,88535.0,92374.0,-7343.0,0.0,173566.0,359784.0
2022-06-30,104757.0,44261.0,3742.0,0.0,16924.0,169684.0,74398.0,6891.0,78822.0,21897.0,195156.0,364840.0,95082.0,47032.0,41595.0,103216.0,198298.0,86939.0,84281.0,-4678.0,0.0,166542.0,364840.0
2022-03-31,104693.0,32613.0,3296.0,0.0,13320.0,153922.0,70298.0,6907.0,78719.0,21845.0,190685.0,344607.0,77439.0,48177.0,41637.0,104244.0,181683.0,85767.0,79633.0,-2476.0,0.0,162924.0,344607.0
2021-12-31,125369.0,33520.0,3019.0,0.0,12280.0,174188.0,67214.0,6994.0,58383.0,21256.0,166201.0,340389.0,77510.0,48260.0,40868.0,102869.0,180379.0,84528.0,75045.0,437.0,0.0,160010.0,340389.0


In [21]:
assets_stackplot = create_stacker_bar(
    balance[
        [
            "Cash On Hand",
            "Receivables",
            "Inventory",
            "Pre-Paid Expenses",
            "Other Current Assets",
            "Property, Plant, And Equipment",
            "Long-Term Investments",
            "Goodwill And Intangible Assets",
            "Other Long-Term Assets",
        ]
    ],
    title_="Total assets",
)


In [22]:
liabilities_stackplot = create_stacker_bar(
    balance[
        [
            "Total Current Liabilities",
            "Long Term Debt",
            "Other Non-Current Liabilities",
            "Total Long Term Liabilities",
            "Common Stock Net",
            "Retained Earnings (Accumulated Deficit)",
            "Comprehensive Income",
            "Other Share Holders Equity",
        ]
    ],
    title_="Total liabilities",
)


In [23]:
# income statement
# income_df = get_macrotrends_data(f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/income-statement?freq=Q")
# income_df.head()

In [24]:
# cash flow statement
cash_flow_df = get_macrotrends_data(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/cash-flow-statement?freq=Q"
)
cash_flow_df.head()


field_name,Net Income/Loss,Total Depreciation And Amortization - Cash Flow,Other Non-Cash Items,Total Non-Cash Items,Change In Accounts Receivable,Change In Inventories,Change In Accounts Payable,Change In Assets/Liabilities,Total Change In Assets/Liabilities,Cash Flow From Operating Activities,"Net Change In Property, Plant, And Equipment",Net Change In Intangible Assets,Net Acquisitions/Divestitures,Net Change In Short-term Investments,Net Change In Long-Term Investments,Net Change In Investments - Total,Investing Activities - Other,Cash Flow From Investing Activities,Net Long-Term Debt,Net Current Debt,Debt Issuance/Retirement Net - Total,Net Common Equity Issued/Repurchased,Net Total Equity Issued/Repurchased,Total Common And Preferred Stock Dividends Paid,Financial Activities - Other,Cash Flow From Financial Activities,Net Cash Flow,Stock-Based Compensation,Common Stock Dividends Paid
2022-12-31,16425.0,3648.0,-3739.0,-91.0,-3164.0,1305.0,-2058.0,1619.0,-5161.0,11173.0,-6274.0,0.0,-679.0,0.0,104.0,104.0,-301.0,-7150.0,-750.0,0.0,-750.0,-5216.0,-5216.0,-5066.0,-317.0,-11349.0,-7238.0,2538.0,-5066.0
2022-09-30,17556.0,2790.0,-2343.0,447.0,11729.0,-543.0,-1567.0,-4834.0,5195.0,23198.0,-6283.0,0.0,-349.0,0.0,4360.0,4360.0,-860.0,-3132.0,-1000.0,0.0,-1000.0,-4998.0,-4998.0,-4621.0,-264.0,-10883.0,8953.0,2192.0,-4621.0
2022-06-30,16740.0,3979.0,14983.0,18962.0,-12634.0,-461.0,2659.0,354.0,-11073.0,24629.0,-6871.0,0.0,-1263.0,0.0,-457.0,-457.0,-1138.0,-9729.0,0.0,0.0,0.0,-8296.0,-8296.0,-4632.0,-341.0,-13269.0,1433.0,1997.0,-4632.0
2022-03-31,16728.0,3773.0,1604.0,5377.0,857.0,-279.0,520.0,1092.0,3281.0,25386.0,-5340.0,0.0,-18719.0,0.0,9069.0,9069.0,-1181.0,-16171.0,-4197.0,0.0,-4197.0,-8345.0,-8345.0,-4645.0,-158.0,-17345.0,-8106.0,1906.0,-4645.0
2021-12-31,18765.0,3496.0,-2570.0,926.0,-5543.0,394.0,235.0,1760.0,-5211.0,14480.0,-5865.0,0.0,-850.0,0.0,5643.0,5643.0,-89.0,-1161.0,0.0,0.0,0.0,-7142.0,-7142.0,-4652.0,-192.0,-11986.0,1439.0,1897.0,-4652.0


In [25]:
compensation_plot = create_plot_bar_line(
    cash_flow_df, "Stock-Based Compensation", bar_color="#EADA52"
)


In [26]:
# financial ratios
fin_ratios_df = get_macrotrends_data(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/financial-ratios?freq=Q"
)
fin_ratios_df.head()


field_name,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2022-12-31,1.9313,0.1941,0.2627,66.8455,38.6733,38.6733,0.0,38.5595,31.1392,0.1447,5.8685,1.4720,61.1404,8.9687,15.6612,4.5055,7.2276,24.5919,1.5001,0.6591
2022-09-30,1.8402,0.2072,0.2801,69.1712,42.9313,42.9313,0.0,43.0390,35.0265,0.1393,3.6204,1.6024,56.1652,10.1149,18.4220,4.8796,8.0186,23.2756,3.0993,2.2599
2022-06-30,1.7846,0.2202,0.2989,68.3235,39.5913,39.5913,0.0,39.5006,32.2761,0.1422,4.3904,1.1718,76.8050,10.0515,19.0835,4.5883,7.8380,22.3127,3.2597,2.3502
2022-03-31,1.9877,0.2282,0.3064,68.3651,41.2561,41.2561,0.0,40.9036,33.8898,0.1432,4.7376,1.5135,59.4646,10.2674,19.8658,4.8542,7.9242,21.7726,3.3839,2.6708
2021-12-31,2.2473,0.2317,0.3328,67.2131,43.0077,43.0077,0.0,43.5258,36.2763,0.1520,5.6178,1.5432,58.3205,11.7274,18.4646,5.5128,9.0099,21.3347,1.9218,1.1443


In [27]:
returns_plot = create_line_plot(
    fin_ratios_df,
    y=[
        "ROE - Return On Equity",
        "Return On Tangible Equity",
        "ROA - Return On Assets",
        "ROI - Return On Investment",
    ],
    title="Returns",
    perc=False,
)


In [28]:
# Debt/Equity Ratio
de_ratio_plot = create_line_plot(
    fin_ratios_df, y=["Debt/Equity Ratio"], title="Debt/Equity Ratio"
)


In [29]:
# Book Value Per Share
# bv_ratio_plot = create_line_plot(fin_ratios_df, y=['Book Value Per Share'], title='Book Value Per Share')

In [30]:
def get_macrotrends_html(url=""):
    html = requests.get(url).text
    soup = BeautifulSoup(html, "lxml")
    html_table = soup.find("table", {"class": "table"}).prettify()
    df = pd.read_html(html_table)[0]
    df.columns = df.columns.get_level_values(1)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date").sort_index()

    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = pd.to_numeric(
                df[c].str.replace("$", "", regex=False), errors="coerce"
            )
    return df


In [31]:
def create_3_subplots(df: pd.DataFrame, indicators: dict, _title=""):

    # Create a figure with subplots
    fig = make_subplots(
        rows=len(indicators), cols=1, shared_xaxes=True, vertical_spacing=0.02
    )

    for i, k in enumerate(indicators, start=1):
        fig.add_trace(indicators[k](x=df.index, y=df[k], name=k), row=i, col=1)

        # adding a median
        if "ratio" in k.lower():
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=[df[k].median()] * len(df[k]),
                    name=k + " median",
                    line=dict(color="grey", width=4, dash="dash"),
                ),
                row=i,
                col=1,
            )

    # Update the layout to have shared x-axis
    fig.update_layout(
        height=600,
        width=600,
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        hovermode="x unified",
        legend_traceorder="normal",
        title=_title,
        legend=dict(orientation="h", yanchor="bottom", y=-0.2),
        xaxis3=dict(title="Dates"),
    )
    fig.show()

    return fig


In [32]:
pe_ratio_df = get_macrotrends_html(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/pe-ratio"
)
# pe_ratio_plot = create_plot_bar_line(pe_ratio_df, 'TTM Net EPS', 'PE Ratio')

pe_ratio_plot = create_3_subplots(
    df=pe_ratio_df,
    indicators={
        "Stock Price": go.Scatter,
        "TTM Net EPS": go.Bar,
        "PE Ratio": go.Scatter,
    },
    _title="PE Ratio",
)


In [33]:
ps_ratio_df = get_macrotrends_html(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/price-sales"
)
# ps_ratio_plot = create_plot_bar_line(ps_ratio_df, 'TTM Sales per Share', 'Price to Sales Ratio')
ps_ratio_plot = create_3_subplots(
    df=ps_ratio_df,
    indicators={
        "Stock Price": go.Scatter,
        "TTM Sales per Share": go.Bar,
        "Price to Sales Ratio": go.Scatter,
    },
    _title="Price to Sales Ratio",
)


In [34]:
pb_ratio_df = get_macrotrends_html(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/price-book"
)

# pb_ratio_plot = create_plot_bar_line(pb_ratio_df, 'Book Value per Share', 'Price to Book Ratio')

pb_ratio_plot = create_3_subplots(
    df=pb_ratio_df,
    indicators={
        "Stock Price": go.Scatter,
        "Book Value per Share": go.Bar,
        "Price to Book Ratio": go.Scatter,
    },
    _title="Price to Book Ratio",
)


In [35]:
pf_ratio_df = get_macrotrends_html(
    f"https://www.macrotrends.net/stocks/charts/{tickers_macrotrends_dict[STOCK]}/price-fcf"
)
# pf_ratio_plot = create_plot_bar_line(pf_ratio_df, 'TTM FCF per Share', 'Price to FCF Ratio')

pf_ratio_plot = create_3_subplots(
    df=pf_ratio_df,
    indicators={
        "Stock Price": go.Scatter,
        "TTM FCF per Share": go.Bar,
        "Price to FCF Ratio": go.Scatter,
    },
    _title="Price to FCF Ratio",
)


Seeking Alpha radar plot

In [38]:

div_safety_metrics = [
    "div_safety_category",
    "cash_div_payout_ratio_ttm",
    "div_payout_gaap",
    "div_payout_nongaap",
    "div_payout_nongaap_fy1",
    "cf_payout",
    "cf_payout_fy1",
    "fcf_yield_div_yield",
    "div_yield_div_payout",
    "div_coverage_ratio_fy1",
    "int_cover",
    "net_lt_debt_tot_assets",
    "net_lt_debt_ebitda",
    "debt_eq",
    "tot_debt_cap",
    "net_margin",
    "rtn_on_common_equity",
    "cash_from_operations_as_reported",
    "cash_per_share_ttm",
    "log_of_unadjusted_stock_price",
    "capm_alpha_60m",
    "institutional_ownership_level",
    "sustainable_growth_rate",
    "div_grow_rate",
    "dividend_coverage_ratio_ttm",
    "fixed_asset_turnover_ttm",
    "dps_consensus_mean_percent_revisions_down_1_annual_period_fwd",
    "net_asset_to_pension_liabilities_annual",
    "div_safety_category_avg_5y",
    "cash_div_payout_ratio_ttm_avg_5y",
    "div_payout_gaap_avg_5y",
    "div_payout_nongaap_avg_5y",
    "div_payout_nongaap_fy1_avg_5y",
    "cf_payout_avg_5y",
    "cf_payout_fy1_avg_5y",
    "fcf_yield_div_yield_avg_5y",
    "div_yield_div_payout_avg_5y",
    "div_coverage_ratio_fy1_avg_5y",
    "int_cover_avg_5y",
    "net_lt_debt_tot_assets_avg_5y",
    "net_lt_debt_ebitda_avg_5y",
    "debt_eq_avg_5y",
    "tot_debt_cap_avg_5y",
    "net_margin_avg_5y",
    "rtn_on_common_equity_avg_5y",
    "cash_from_operations_as_reported_avg_5y",
    "cash_per_share_ttm_avg_5y",
    "log_of_unadjusted_stock_price_avg_5y",
    "capm_alpha_60m_avg_5y",
    "institutional_ownership_level_avg_5y",
    "sustainable_growth_rate_avg_5y",
    "div_grow_rate_avg_5y",
    "dividend_coverage_ratio_ttm_avg_5y",
    "fixed_asset_turnover_ttm_avg_5y",
    "dps_consensus_mean_percent_revisions_down_1_annual_period_fwd_avg_5y",
    "net_asset_to_pension_liabilities_annual_avg_5y",
]

div_growth_metrics = [
    "div_growth_category",
    "dps_yoy",
    "dividend_per_share_change_dislpay",
    "dividend_lt_fwd_growth",
    "div_grow_rate3",
    "div_grow_rate5",
    "div_grow_rate10",
    "revenue_change_display",
    "eps_change_display",
    "fcf_per_share_change_display",
    "ebitda_change_display",
    "ebit_change_display",
    "return_on_net_tangible_assets",
    "log_of_unadjusted_stock_price",
    "coefficient_of_variation_90d",
    "degree_of_operating_leverage_ttm",
    "div_growth_category_avg_5y",
    "dps_yoy_avg_5y",
    "dividend_per_share_change_dislpay_avg_5y",
    "dividend_lt_fwd_growth_avg_5y",
    "div_grow_rate3_avg_5y",
    "div_grow_rate5_avg_5y",
    "div_grow_rate10_avg_5y",
    "revenue_change_display_avg_5y",
    "eps_change_display_avg_5y",
    "fcf_per_share_change_display_avg_5y",
    "ebitda_change_display_avg_5y",
    "ebit_change_display_avg_5y",
    "return_on_net_tangible_assets_avg_5y",
    "log_of_unadjusted_stock_price_avg_5y",
    "coefficient_of_variation_90d_avg_5y",
    "degree_of_operating_leverage_ttm_avg_5y",
]

div_yield_metrics = [
    "div_yield_category",
    "div_yield_4y",
    "dividend_yield",
    "div_yield_fwd",
    "yld_on_cost_1y",
    "yld_on_cost_3y",
    "yld_on_cost_5y",
    "earnings_yield",
    "earn_yield_gaap_fy1",
    "oper_income_market_cap",
    "oper_income_fy1_market_cap",
    "fcf_yield",
    "fcf_yield_fy1",
    "div_yield_category_avg_5y",
    "div_yield_4y_avg_5y",
    "dividend_yield_avg_5y",
    "div_yield_fwd_avg_5y",
    "yld_on_cost_1y_avg_5y",
    "yld_on_cost_3y_avg_5y",
    "yld_on_cost_5y_avg_5y",
    "earnings_yield_avg_5y",
    "earn_yield_gaap_fy1_avg_5y",
    "oper_income_market_cap_avg_5y",
    "oper_income_fy1_market_cap_avg_5y",
    "fcf_yield_avg_5y",
    "fcf_yield_fy1_avg_5y",
]

div_history_metrics = [
    "div_consistency_category",
    "dividend_growth",
    "dividend_consistency",
    "div_consistency_category_avg_5y",
    "dividend_growth_avg_5y",
    "dividend_consistency_avg_5y",
]

earnings_metrics = [
    "analysts_up_percent",
    "analysts_down_percent",
    "analysts_up",
    "analysts_down",
    "analysts_up_percent_avg_5y",
    "analysts_down_percent_avg_5y",
    "analysts_up_avg_5y",
    "analysts_down_avg_5y",
]

valuation_metrics = [
    "pe_nongaap",
    "pe_nongaap_fy1",
    "pe_ratio",
    "pe_gaap_fy1",
    "peg_gaap",
    "peg_nongaap_fy1",
    "ev_12m_sales_ratio",
    "ev_sales_fy1",
    "ev_ebitda",
    "ev_ebitda_fy1",
    "ev_ebit",
    "ev_ebit_fy1",
    "ps_ratio",
    "ps_ratio_fy1",
    "pb_ratio",
    "pb_fy1_ratio",
    "price_cf_ratio",
    "price_cf_ratio_fy1",
    "dividend_yield",
    "pe_nongaap_avg_5y",
    "pe_nongaap_fy1_avg_5y",
    "pe_ratio_avg_5y",
    "pe_gaap_fy1_avg_5y",
    "peg_gaap_avg_5y",
    "peg_nongaap_fy1_avg_5y",
    "ev_12m_sales_ratio_avg_5y",
    "ev_sales_fy1_avg_5y",
    "ev_ebitda_avg_5y",
    "ev_ebitda_fy1_avg_5y",
    "ev_ebit_avg_5y",
    "ev_ebit_fy1_avg_5y",
    "ps_ratio_avg_5y",
    "ps_ratio_fy1_avg_5y",
    "pb_ratio_avg_5y",
    "pb_fy1_ratio_avg_5y",
    "price_cf_ratio_avg_5y",
    "price_cf_ratio_fy1_avg_5y",
    "dividend_yield_avg_5y",
]

growth_metrics = [
    "revenue_growth",
    "revenue_change_display",
    "ebitda_yoy",
    "ebitda_change_display",
    "operating_income_ebit_yoy",
    "ebit_change_display",
    "diluted_eps_growth",
    "eps_change_display",
    "eps_ltg",
    "levered_free_cash_flow_yoy",
    "fcf_per_share_change_display",
    "op_cf_yoy",
    "cf_op_change_display",
    "roe_yoy",
    "roe_change_display",
    "working_cap_change",
    "capex_change",
    "dividend_per_share_change_dislpay",
    "dps_yoy",
    "revenue_growth_avg_5y",
    "revenue_change_display_avg_5y",
    "ebitda_yoy_avg_5y",
    "ebitda_change_display_avg_5y",
    "operating_income_ebit_yoy_avg_5y",
    "ebit_change_display_avg_5y",
    "diluted_eps_growth_avg_5y",
    "eps_change_display_avg_5y",
    "eps_ltg_avg_5y",
    "levered_free_cash_flow_yoy_avg_5y",
    "fcf_per_share_change_display_avg_5y",
    "op_cf_yoy_avg_5y",
    "cf_op_change_display_avg_5y",
    "roe_yoy_avg_5y",
    "roe_change_display_avg_5y",
    "working_cap_change_avg_5y",
    "capex_change_avg_5y",
    "dividend_per_share_change_dislpay_avg_5y",
    "dps_yoy_avg_5y",
]

growth_symbol_data_fields = [
    "revenue_growth",
    "revenue_growth3",
    "revenue_growth5",
    "revenueGrowth10",
    "ebitdaYoy",
    "ebitda_3y",
    "ebitda_5y",
    "ebitda_10y",
    "operatingIncomeEbitYoy",
    "operatingIncomeEbit3y",
    "operatingIncomeEbit5y",
    "operatingIncomeEbit10y",
    "netIncomeYoy",
    "netIncome3y",
    "netIncome5y",
    "netIncome10y",
    "normalizedNetIncomeYoy",
    "normalizedNetIncome3y",
    "normalizedNetIncome5y",
    "normalizedNetIncome10y",
    "earningsGrowth",
    "earningsGrowth3",
    "earningsGrowth5y",
    "earningsGrowth10y",
    "dilutedEpsGrowth",
    "dilutedEps3y",
    "dilutedEps5y",
    "dilutedEps10y",
    "tangibleBookValueYoy",
    "tangibleBookValue3y",
    "tangibleBookValue5y",
    "tangibleBookValue10y",
    "totalAssetsYoy",
    "totalAssets3y",
    "totalAssets5y",
    "totalAssets10y",
    "leveredFreeCashFlowYoy",
    "leveredFreeCashFlow3y",
    "leveredFreeCashFlow5y",
    "leveredFreeCashFlow10y",
    "net_interest_income_yoy",
    "net_interest_income_3y",
    "net_interest_income_5y",
    "net_interest_income_10y",
    "gross_loans_yoy",
    "gross_loans_3y",
    "gross_loans_5y",
    "gross_loans_10y",
    "common_equity_yoy",
    "common_equity_3y",
    "common_equity_5y",
    "common_equity_10y",
]

profitability_metrics = [
    "gross_margin",
    "ebit_margin",
    "ebitda_margin",
    "net_margin",
    "levered_fcf_margin",
    "rtn_on_common_equity",
    "return_on_total_capital",
    "return_on_avg_tot_assets",
    "capex_to_sales",
    "assets_turnover",
    "cash_from_operations_as_reported",
    "net_inc_per_employee",
    "gross_margin_avg_5y",
    "ebit_margin_avg_5y",
    "ebitda_margin_avg_5y",
    "net_margin_avg_5y",
    "levered_fcf_margin_avg_5y",
    "rtn_on_common_equity_avg_5y",
    "return_on_total_capital_avg_5y",
    "return_on_avg_tot_assets_avg_5y",
    "capex_to_sales_avg_5y",
    "assets_turnover_avg_5y",
    "cash_from_operations_as_reported_avg_5y",
    "net_inc_per_employee_avg_5y",
]

peers_metrics = [
    "marketcap_display",
    "tev",
    "number_of_employees",
    "authors_count",
    "tot_analysts_recommendations",
    "close",
    "price_high_52w",
    "price_low_52w",
    "p_week_vol_shares",
    "total_return_1m",
    "total_return_3m",
    "total_return_6m",
    "total_return_9m",
    "total_return_ytd",
    "total_return_1y",
    "total_return_3y",
    "total_return_5y",
    "total_return_10y",
    "total_cash",
    "cash_per_share",
    "total_debt",
    "net_debt",
    "debt_eq",
    "debt_short_term",
    "debt_long_term",
    "current_ratio",
    "quick_ratio",
    "interest_coverage_ratio",
    "book_value",
    "debt_fcf",
    "long_term_debt_per_capital",
    "nocf",
    "cash_from_operations_as_reported",
    "levered_free_cash_flow",
    "capital_expenditures",
]

wallstreet_metrics = [
    'authors_rating_strong_buy_count',
    'authors_rating_buy_count',
    'authors_rating_hold_count',
    'authors_rating_sell_count',
    'authors_rating_strong_sell_count',
    'sell_side_rating_strong_buy_count',
    'sell_side_rating_buy_count',
    'sell_side_rating_hold_count',
    'sell_side_rating_sell_count',
    'sell_side_rating_strong_sell_count',
    'authors_rating',
    'sell_side_rating',
]

seeking_alpha_all_metrics = {
    'Dividend safety' : div_safety_metrics,
    'Dividend growth' : div_growth_metrics,
    'Dividend yield' : div_yield_metrics,
    'Dividend history' : div_history_metrics,
    'Earnings' : earnings_metrics,
    'Valuation' : valuation_metrics,
    'Growth' : growth_metrics+growth_symbol_data_fields,
    'Profitability' : profitability_metrics,
    'Wallstreet rating' : wallstreet_metrics,
    }

In [101]:
def get_data_from_seeking_alpha(metrics_list:list, method=''):

    headers = {
        "cookie": "machine_cookie=9717268612629; machine_cookie_ts=1671790378",
        "authority": "seekingalpha.com",
        "referer": f"https://seekingalpha.com/symbol/{STOCK}/dividends/dividend-growth",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    }
    result = {}

    if method =='grades':
        url = "https://seekingalpha.com/api/v3/ticker_metric_grades"
        querystring = {
            "filter[fields][]": metrics_list,
            "filter[slugs]": f"{STOCK.lower()}",
            "filter[algos][]": ["main_quant", "dividends"],
            "minified": "false",
        }
        response = requests.request(
            "GET", url, headers=headers, params=querystring
        ).json()
        for item in response["data"]:
            grade = item["attributes"]["grade"]
            metric_id = item["relationships"]["metric_type"]["data"]["id"]
            for included in response["included"]:
                if included["id"] == metric_id:
                    result[included["attributes"]["field"]] = grade
    
    elif method =='sector':
        url = f"https://seekingalpha.com/api/v3/symbols/{STOCK.lower()}/sector_metrics"
        querystring = {"filter[fields][]": metrics_list}
        response = requests.request(
            "GET", url, headers=headers, params=querystring
        ).json()
        for item in response["data"]:
            value = item["attributes"]["value"]
            metric_id = item["relationships"]["metric_type"]["data"]["id"]
            for included in response["included"]:
                if included["id"] == metric_id:
                    result[included["attributes"]["field"]] = value

    else:
        url = "https://seekingalpha.com/api/v3/metrics"
        querystring = {
            "filter[fields][]": metrics_list,
            "filter[slugs]": f"{STOCK.lower()}",
            "minified": "false",
        }
        
        response = requests.request(
            "GET", url, headers=headers, params=querystring
        ).json()
        for item in response["data"]:
            field = next(
                filter(
                    lambda included: included["id"]
                    == item["relationships"]["metric_type"]["data"]["id"],
                    response["included"],
                )
            )["attributes"]["field"]
            result[field] = item["attributes"]["value"]
            
    return result


In [102]:
def create_seeking_alpha_df(field='All'):
    if field=='All':
        metrics_list = []
        for v in seeking_alpha_all_metrics.values():
            for m in v:
                metrics_list.append(m)
    else:
        metrics_list = seeking_alpha_all_metrics[field]

    metrics = get_data_from_seeking_alpha(metrics_list, "metrics")

    averages = {}    
    for k in list(metrics.keys()):
        if '_avg_5y' in k:
            averages[k] = metrics[k]
            del metrics[k]
    new_keys = {}

    for k, v in list(averages.items()):
        if '_avg_5y' in k:
            new_keys[k] = k.replace("_avg_5y","")

    for old, new in new_keys.items():
        averages[new] = averages.pop(old)

    sector = get_data_from_seeking_alpha(metrics_list, "sector")
    grades = get_data_from_seeking_alpha(metrics_list, "grades")
    

    grades_dict = {
        1: "A+",
        2: "A",
        3: "A-",
        4: "B+",
        5: "B",
        6: "B-",
        7: "C+",
        8: "C",
        9: "C-",
        10: "D+",
        11: "D",
        12: "D-",
        13: "E+",
        14: "E",
        15: "E-",
        16: "F+",
        17: "F",
        18: "F-",
    }

    seeking_alpha_df = pd.DataFrame(
        [metrics, averages, sector, grades], index=["ticker", "avg_5y", "sector", "grade"]
    ).T.dropna()
    seeking_alpha_df['diff_sector'] = seeking_alpha_df["ticker"] / seeking_alpha_df["sector"]
    seeking_alpha_df['diff_avg_5y'] = seeking_alpha_df["ticker"] / seeking_alpha_df["avg_5y"]
    seeking_alpha_df["grade_final"] = seeking_alpha_df["grade"].map(grades_dict)

    return seeking_alpha_df

In [103]:
seeking_alpha_df = []
for k in seeking_alpha_all_metrics.keys():
    df = create_seeking_alpha_df(k)
    df['field'] = k
    seeking_alpha_df.append(df)

seeking_alpha_df = pd.concat(seeking_alpha_df)
seeking_alpha_df

,ticker,avg_5y,sector,grade,diff_sector,diff_avg_5y,grade_final,field
net_margin,3.304800e+01,3.026851e+01,3.245900e+00,1.0,10.181460,1.091828,A+,Dividend safety
cash_from_operations_as_reported,8.438600e+10,6.343622e+10,5.203600e+07,1.0,1621.684930,1.330249,A+,Dividend safety
rtn_on_common_equity,3.931210e+01,3.966441e+01,4.971500e+00,1.0,7.907493,0.991118,A+,Dividend safety
div_payout_nongaap,2.785088e+01,3.482918e+01,2.643170e+01,7.0,1.053692,0.799642,C+,Dividend safety
cash_div_payout_ratio_ttm,3.180918e+01,3.364890e+01,3.180920e+01,7.0,1.000000,0.945326,C+,Dividend safety
...,...,...,...,...,...,...,...,...
cash_from_operations_as_reported,8.438600e+10,6.343622e+10,5.203600e+07,1.0,1621.684930,1.330249,A+,Profitability
net_inc_per_employee,3.051991e+05,2.823485e+05,4.737960e+03,1.0,64.415719,1.080930,A+,Profitability
rtn_on_common_equity,3.931210e+01,3.966441e+01,4.971500e+00,1.0,7.907493,0.991118,A+,Profitability
return_on_total_capital,2.084180e+01,1.742308e+01,3.045800e+00,1.0,6.842800,1.196218,A+,Profitability


In [117]:
# @st.cache
def create_radar_plot(df_orig: pd.DataFrame, field='', value="grade"):
    """
    Dividend safety \n
    Dividend growth \n
    Dividend yield \n
    Dividend history \n
    Earnings \n
    Valuation \n
    Growth \n
    Profitability \n
    Wallstreet rating
    """

    if field!='':
        df = df_orig.loc[df_orig['field']==field].copy()
    else:
        df = df_orig.copy()

    fig = go.Figure()

    fig.add_trace(
        go.Scatterpolar(
            r=df[value],  # Radial coordinates of each point
            theta=df.index,  # Angular coordinates of each point
            fill="toself",  # Fill the area between the line and the radial axis
            name=value,
            # hovertemplate=[
            #     f'{i}: {df.at[i, "ticker"]:.2%}' for i in df.index
            # ],  # Show ticker on hover
            marker=dict(color="#1fd655"),  # Change color of the plot
        )
    )

    r = np.ones(len(df.index))
    if value=='grade':
        r = r*7

    fig.add_trace(
        go.Scatterpolar(
            r=r,
            theta=df.index,
            name=value,
            # hovertemplate=[
            #     f'{i}: {df.at[i, "sector"]:.2%}' for i in df.index
            # ],  # Show sector on hover
            marker=dict(color="red"),  # Change color of the plot
        )
    )

    # Define the layout of the plot
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True),
            angularaxis=dict(
                direction="clockwise"
            ),  # Set the direction of the angular axis
        ),
        showlegend=False,
        width=800,
        height=500,
        margin=dict(l=200, r=200, t=20, b=20),
    )
    fig.for_each_trace(lambda t: t.update(hoveron="points"))

    fig.show()
    return fig


grades_radar_plot = create_radar_plot(seeking_alpha_df, value="grade", field='Dividend yield')


EPS estimate plot

In [39]:
# earning_history = yq.Ticker(STOCK).earning_history
earning_history = yf.Ticker(STOCK).earnings_history.iloc[:, 2:]

earning_history["Earnings Date"] = pd.to_datetime(
    [" ".join(e.split(",")[:-1]) for e in earning_history["Earnings Date"]]
)

earning_history["Surprise(%)"] = [
    float(s.replace("+", "")) / 100 if type(s) == str else s
    for s in earning_history["Surprise(%)"]
]

earning_history["EPS Difference"] = (
    earning_history["Reported EPS"] - earning_history["EPS Estimate"]
)

earning_history = (
    earning_history.set_index("Earnings Date")
    .dropna(how="all", axis=0)
    .drop_duplicates()
)

earning_history["Surprise_abs"] = np.abs(earning_history["Surprise(%)"]).fillna(0)

earning_history


,EPS Estimate,Reported EPS,Surprise(%),EPS Difference,Surprise_abs
Earnings Date,,,,,
2023-01-24,2.29,NaN,NaN,NaN,0.0000
2022-10-25,2.30,2.35,0.0204,0.05,0.0204
2022-07-26,2.29,2.23,-0.0275,-0.06,0.0275
2022-04-26,2.19,2.22,0.0160,0.03,0.0160
2022-01-25,2.31,2.48,0.0727,0.17,0.0727
...,...,...,...,...,...
2000-07-18,0.21,0.22,0.0581,0.01,0.0581
2000-04-20,0.20,0.22,0.0527,0.02,0.0527
2000-01-18,0.21,0.24,0.1127,0.03,0.1127


In [108]:
def create_eps_estimate_plot(df=earning_history, size=5, limit=False):
    if limit:
        df = df.loc[
            df.index
            >= (dt.date.today() - dt.timedelta(days=365 * 3)).strftime("%Y-%m-%d")
        ]
        size = 20  # earning_history['Surprise_abs']*100

    hover_text = df[["Surprise(%)", "EPS Difference"]].apply(
        lambda x: "Surprise (%): {:.2%} <br>EPS Difference: {:.2f}".format(x[0], x[1]),
        axis=1,
    )

    epsActual_trace = go.Scatter(
        x=df.index,
        y=df["Reported EPS"],
        name="Reported EPS",
        mode="markers",
        marker=dict(
            size=size,
            color=[
                "green"
                if df["Reported EPS"][i] > df["EPS Estimate"][i]
                else "red"
                if df["Reported EPS"][i] < df["EPS Estimate"][i]
                else "grey"
                for i in range(len(df))
            ],
        ),
    )

    epsEstimate_trace = go.Scatter(
        x=df.index,
        y=df["EPS Estimate"],
        name="EPS Estimate",
        mode="markers",
        text=hover_text,
        marker=dict(color="grey", size=size),
    )

    data = [epsActual_trace, epsEstimate_trace]
    layout = go.Layout(title="EPS Estimates")
    fig = go.Figure(data=data, layout=layout)

    fig.update_layout(
        height=400,
        width=600,
        margin=dict(l=20, r=20, t=30, b=20),
        template="plotly_dark",
        hovermode="x unified",
        barmode="relative",
        legend=dict(
            orientation="h", yanchor="bottom", y=-0.2, xanchor="left", title=None
        ),
        # showlegend=False
    )

    fig.show()


create_eps_estimate_plot(limit=True)


Recommendation plot

In [41]:
recommendation_df = yq.Ticker(STOCK).recommendation_trend.reset_index(drop=True)
for i, p in enumerate(recommendation_df["period"]):
    recommendation_df.loc[i, "date"] = dt.date.today() + dt.timedelta(
        days=30 * int(recommendation_df.loc[i, "period"].replace("m", ""))
    )
recommendation_df["date"] = [d.strftime("%Y-%m") for d in recommendation_df["date"]]
recommendation_df = recommendation_df.set_index("date").drop(["period"], axis=1)
recommendation_df


,strongBuy,buy,hold,sell,strongSell
date,,,,,
2023-01,14,13,6,0,1
2022-12,16,29,5,0,0
2022-11,17,31,5,0,0
2022-10,18,30,3,0,0


In [117]:
colors = [
    "#ff6962",
    "#ff8989",
    "#ffb3a5",
    "#77dd76",
    "#03c03c",
]  # '#5fa777' 7abd91 # colors=px.colors.sequential.Rainbow
data = []

for i, column in enumerate(recommendation_df.columns[::-1]):
    data.append(
        go.Bar(
            x=recommendation_df.index,
            y=recommendation_df[column],
            name=column,
            text=recommendation_df[column],
            textfont={"color": "black"},
            textposition="inside",
            marker=dict(color=colors[i]),
        )
    )

layout = go.Layout(title="Recommendations", barmode="stack")

fig = go.Figure(data=data, layout=layout)

fig.update_layout(
    height=400,
    width=400,
    margin=dict(l=20, r=20, t=30, b=20),
    template="plotly_dark",
    hovermode="x unified",
)
fig.show()


In [123]:
prices_df = pd.DataFrame(
    yf.download(
        stock_list_test,
        (dt.date.today() - dt.timedelta(days=365 * 2)).strftime("%Y-%m-%d"),
    )["Adj Close"]
)
prices_df["rolling_max"] = prices_df["Adj Close"].rolling(window=252).max()
prices_df["rolling_min"] = prices_df["Adj Close"].rolling(window=252).min()
prices_df["rolling_avg"] = prices_df["Adj Close"].rolling(window=252).mean()
prices_df = prices_df.dropna(axis=0)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=prices_df["rolling_min"], x=prices_df.index, mode="lines", name="Minimum 52w"
    )
)
fig.add_trace(
    go.Scatter(
        y=prices_df["rolling_max"], x=prices_df.index, mode="lines", name="Maximum 52w"
    )
)
fig.add_trace(
    go.Scatter(
        y=prices_df["Adj Close"], x=prices_df.index, mode="lines", name="Current"
    )
)
fig.add_trace(
    go.Scatter(
        y=prices_df["rolling_avg"],
        x=prices_df.index,
        mode="lines",
        name="Average 52w",
        line=dict(color="grey", width=4, dash="dash"),
    )
)

fig.update_layout(
    title="52 Week Price Range",
    yaxis_title="Price",
    height=400,
    width=600,
    margin=dict(l=20, r=20, t=30, b=20),
    template="plotly_dark",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="left", title=None),
)
fig.show()


[*********************100%***********************]  1 of 1 completed


In [109]:
get_data_from_seeking_alpha(div_estimate_metrics, method='')

{'dividends_estimate_fy1_analyst_down': 4.0,
 'dividends_estimate_fy1_analyst_up': 2.0,
 'dividends_estimate_fy2_analyst_down': 3.0,
 'dividends_estimate_fy2_analyst_up': 1.0}

In [ ]:
div_estimate_metrics = [
    "dividends_estimate_fy1_analyst_up_avg_5y",
    "dividends_estimate_fy1_analyst_down_avg_5y",
    "dividends_estimate_fy2_analyst_up_avg_5y",
    "dividends_estimate_fy2_analyst_down_avg_5y",
    "dividends_estimate_fy1_analyst_up",
    "dividends_estimate_fy1_analyst_down",
    "dividends_estimate_fy2_analyst_up",
    "dividends_estimate_fy2_analyst_down",
]


In [ ]:
div_estimate_tooltips = "%2Fsymbol%2FDPZ%2Fdividends%2Festimates"
div_safety_tooltips = "%2Fsymbol%2FDPZ%2Fdividends%2Fdividend-safety"
div_growth_tooltips = "%2Fsymbol%2FDPZ%2Fdividends%2Fdividend-growth"
div_history_tooltips = "%2Fsymbol%2FDPZ%2Fdividends%2Fhistory"
div_yield_tooltips = "%2Fsymbol%2FDPZ%2Fdividends%2Fyield"
valuation_tooltips = "%2Fsymbol%2FDPZ%2Fvaluation%2Fmetrics"
growth_tooltips = "%2Fsymbol%2FDPZ%2Fgrowth"
profitability_tooltips = "%2Fsymbol%2FDPZ%2Fprofitability"
earnings_tooltips = "%2Fsymbol%2FDPZ%2Fearnings"
peers_tooltips = "%2Fsymbol%2FDPZ%2Fpeers%2Fcomparison"
# &filter[slugs]=dpz,armk,dri,txrh,bros,wen&minified=false
"""quant rating"""
quant_rating_tooltips="%2Fsymbol%2FDPZ%2Fratings%2Fquant-ratings"
# https://seekingalpha.com/api/v3/rating/periods[periods]=[0&filter[periods]=[3&filter[periods]=[6
# https://seekingalpha.com/api/v3/rating/histories?page[number]=1
# https://seekingalpha.com/api/v3/historical_prices[ticker][slug]=dpz&&filter[as_of_date][gte]=2022-07-25&filter[as_of_date][lte]=2022-09-12&sort=as_of_date
# https://seekingalpha.com/api/v3/fundamentals_metrics?period_type=annual&statement_type=income-statement&target_currency=USD

"""wall street ratings"""
wallstreat_tooltips="%2Fsymbol%2FDPZ%2Fratings%2Fsell-side-ratings"
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=outperform,buy,hold,underperform,sell&group_by_month=true&period_type=non_periodic&return_window=4&ticker_ids=2557
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=target_price&group_by_month=false&period_type=non_periodic&return_window=4&ticker_ids=2557
# https://seekingalpha.com/api/v3/symbol_data/estimates?estimates_data_items=target_price,target_price_high,target_price_low&group_by_month=false&period_type=non_periodic&return_window=1&ticker_ids=2557
# https://seekingalpha.com/api/v3/rating/periods[periods]=[0&filter[periods]=[3&filter[periods]=[6

"""nasdaq analyst"""
# https://www.nasdaq.com/market-activity/stocks/dpz/analyst-research

In [43]:
"""streamlit - we need .py file, not .ipynb!"""
# st.plotly_chart(dividends_radar_plot)

# def show_streamlit_plot(fig):
# st.plotly_chart(fig)

# show_streamlit_plot(dividends_radar_plot)

# streamlit run c:\Users\nazaire1703\miniconda3\lib\site-packages\ipykernel_launcher.py
# RuntimeError: There is no current event loop in thread 'ScriptRunner.scriptThread'.
# AssertionError: init_sockets cannot be called twice!


'streamlit - we need .py file, not .ipynb!'

In [44]:
"""dividend_cagrs"""
# url = "https://seekingalpha.com/api/v3/symbol_data"
# querystring = {"fields[]":"dividend_cagrs", "slugs":f"{STOCK.lower()}"}
# response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
# result = response['data'][0]['attributes'] #['dividendCagrs']
# result

# ticker.technical_insights

'dividend_cagrs'

In [224]:
""" dividend yield tooltips"""

# url = "https://seekingalpha.com/api/v3/tooltips"

# querystring = {"filter[path]" : f"/symbol/{STOCK}"}
# response1 = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()

# tooltips = {}

# for item in response1['data']:
#     term = item['attributes']['term']
#     content = item['attributes']['content']
#     response1[term] = content

# querystring = {"filter[path]" : f"/symbol/{STOCK}/dividends/yield"}
# response2 = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()

# for item in response2['data']:
#     term = item['attributes']['term']
#     content = item['attributes']['content']
#     tooltips[term] = content
    
# tooltips

' dividend yield tooltips'

In [225]:
"""articles"""
# url = f"https://seekingalpha.com/api/v3/symbols/{STOCK.lower()}/author_ratings"
# querystring = {"include" : ['article']}
# response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
# result = {}
# for item in response['data']:
#     article_id = item['relationships']['article']['data']['id']
#     for item in response['included']:
#         if item['id'] == article_id:
#             title = item['attributes']['title']
#             pro_publish_on = item['attributes']['proPublishOn']
#             result[pro_publish_on] = title
# result

'articles'

Alpha Spread

In [145]:
"""BeautifulSoup"""
url_ = f"https://www.alphaspread.com/security/nasdaq/aapl/summary"

# Reuse a session object: 
# Instead of creating a new session object for each request, you can reuse the same session object for multiple requests. 
# This can save time by not having to establish a new connection for each request.

# session_object = requests.Session()
# response = session_object.get(url_)

# response = requests.get(url)
# html = response.text
# soup = BeautifulSoup(html, "lxml")
# soup

# https://www.alphaspread.com/security/nasdaq/aapl/summary
# https://www.alphaspread.com/security/nasdaq/aapl/dcf-valuation
# https://www.alphaspread.com/security/nasdaq/aapl/relative-valuation
# https://www.alphaspread.com/security/nasdaq/aapl/analyst-estimates
# https://www.alphaspread.com/security/nasdaq/aapl/profitability
# https://www.alphaspread.com/security/nasdaq/aapl/solvency
# https://www.alphaspread.com/security/nasdaq/aapl/financials/balance-sheet
# https://www.alphaspread.com/security/nasdaq/aapl/financials/income-statement
# https://www.alphaspread.com/security/nasdaq/aapl/financials/cash-flow-statement
# https://www.alphaspread.com/security/nasdaq/aapl/financials/revenue-breakdown
# https://www.alphaspread.com/security/nasdaq/aapl/discount-rate

In [146]:
"""NASDAQ data link"""
# https://docs.data.nasdaq.com/docs/in-depth-usage
# https://data.nasdaq.com/tools/python
f"https://data.nasdaq.com/api/v3/datasets/WIKI/{STOCK}/data.json?api_key={NASDAQ_DATA_LINK_API_KEY}"

"""last date on AAPL is 2018-03-27"""
# url = "https://data.nasdaq.com/api/v3/datasets/WIKI/AAPL/data.json"
# querystring = {"api_key":NASDAQ_DATA_LINK_API_KEY}
# response = requests.request("GET", url, params=querystring).json()
# ndl_df = pd.DataFrame(data=response['dataset_data']['data'], columns=response['dataset_data']['column_names'])
# ndl_df

"""last date on AAPL is 2017-10-31"""
# url = "https://data.nasdaq.com/api/v3/datatables/QUOTEMEDIA/PRICES"
# querystring = {"api_key":NASDAQ_DATA_LINK_API_KEY, 'ticker':'AAPL'}
# response = requests.request("GET", url, params=querystring).json()['datatable']
# ndl_df = pd.DataFrame(data=response['data'], columns=[i['name'] for i in response['columns']])
# ndl_df

"""income from seeking alpha (doesn't work)"""
# url = "https://seekingalpha.com/api/v3/symbols/dpz/fundamentals_metrics"
# querystring = {
#     "period_type" : 'quarterly',
#     'statement_type':'income-statement',
#     'target_currency':'USD'
# }
# response = requests.request("GET", url, headers=headers_div_growth, params=querystring).json()
# response


"income from seeking alpha (doesn't work)"

# Summary

In [78]:
stocks_full_summary = pd.DataFrame()
for x in stock_list_test:
    print(x)
    ticker = yq.Ticker(x)
    # ticker_yf = yf.Ticker(x)

    summary = pd.DataFrame(ticker.summary_detail)
    
    recomendations = ticker.recommendation_trend.reset_index().query('period=="0m"')[['strongBuy','buy','hold','sell','strongSell']].T
    recomendations.columns = [x]
    
    financials = pd.DataFrame(ticker.financial_data)
    key_stats = pd.DataFrame(ticker.key_stats)
    # esg_scores = pd.DataFrame(ticker.esg_scores)
    profile = pd.DataFrame(ticker.summary_profile)

    # earnings
    earnings = ticker.earning_history

    # grades
    grades = ticker.grading_history
    grades['epochGradeDate'] = pd.to_datetime(grades['epochGradeDate'])

    # income
    income_statement = ticker.income_statement().query("periodType=='12M'")

    if 'OperatingIncome' in income_statement.columns:
        income_statement['operatingMargin'] = income_statement['OperatingIncome']/income_statement['TotalRevenue']
    else:
        income_statement['operatingMargin'] = income_statement['PretaxIncome']/income_statement['TotalRevenue']
    
    if 'GrossProfit' in income_statement.columns:
        income_statement['grossMargin'] = income_statement['GrossProfit']/income_statement['TotalRevenue']
    else:
        income_statement['grossMargin'] = np.nan
    income_statement['avgGrossMarginGrowth'] = income_statement['grossMargin'].diff()
    income_statement['avgOperatingMarginGrowth'] = income_statement['operatingMargin'].diff()
    income_statement['avgNetIncomeGrowth'] = income_statement['NetIncome'].diff()/income_statement['NetIncome'].shift(1)
    income_statement['avgTotalRevenueGrowth'] = income_statement['TotalRevenue'].diff()/income_statement['TotalRevenue'].shift(1)

    # balance
    balance = ticker.balance_sheet().query('periodType=="12M"')
    balance['avgTotalAssetsGrowth'] = balance['TotalAssets'].diff()/balance['TotalAssets'].shift(1)
    balance['avgTotalDebtGrowth'] = balance['TotalDebt'].diff()/balance['TotalDebt'].shift(1)

    if 'NetDebt' in balance.columns:
        balance['avgNetDebtGrowth'] = balance['NetDebt'].diff()/balance['NetDebt'].shift(1)
    elif 'CurrentDebt' in balance.columns:
        balance['avgNetDebtGrowth'] = balance['CurrentDebt'].diff()/balance['CurrentDebt'].shift(1)
    else:
        balance['avgNetDebtGrowth'] = np.nan

    # cash flow
    cashflow = ticker.cash_flow().query('periodType=="12M"')
    cashflow['avgFreeCashFlowGrowth'] = cashflow['FreeCashFlow'].diff()/cashflow['FreeCashFlow'].shift(1)

    # final row
    row = pd.concat([profile, summary, key_stats, financials, recomendations], axis=0)
    row = row[~row.index.duplicated(keep='first')].T

    row['longName'] = ticker.price[x]['longName']
    if type(earnings['surprisePercent'][0]) != dict:
        row['avgSurprice'] = earnings['surprisePercent'].median()
        row['lastSurprice'] = earnings.iloc[-1]['surprisePercent']
        row['earning_dynamics'] = earnings.iloc[0]['epsActual'] / earnings.iloc[-1]['epsActual']
    else:
        row['avgSurprice'] = 0
        row['lastSurprice'] = 0
        row['earning_dynamics'] = 0

    row['operatingMargin'] = income_statement.iloc[-1]['operatingMargin']
    row['grossMargin'] = income_statement.iloc[-1]['grossMargin']
    row['operatingMarginStability'] = income_statement['operatingMargin'].max() - income_statement['operatingMargin'].min()
    row['avgGrossMarginGrowth'] = income_statement['avgGrossMarginGrowth'].mean()
    row['avgOperatingMarginGrowth'] = income_statement['avgOperatingMarginGrowth'].mean()
    row['avgNetIncomeGrowth'] = income_statement['avgNetIncomeGrowth'].mean()
    row['avgTotalRevenueGrowth'] = income_statement['avgTotalRevenueGrowth'].mean()
    row['netIncomeStability'] = income_statement['avgNetIncomeGrowth'].max() - income_statement['avgNetIncomeGrowth'].min()

    row['avgTotalAssetsGrowth'] = balance['avgTotalAssetsGrowth'].mean()
    row['avgTotalDebtGrowth'] = balance['avgTotalDebtGrowth'].mean()
    row['avgNetDebtGrowth'] = balance['avgNetDebtGrowth'].mean()
    row['netDebtStability'] = balance['avgNetDebtGrowth'].max() - balance['avgNetDebtGrowth'].min()

    if 'NetDebt' in balance.columns:
        row['lastNetDebt'] = balance.iloc[-1]['NetDebt']
    elif 'CurrentDebt' in balance.columns:
        row['lastNetDebt'] = balance.iloc[-1]['CurrentDebt']
    else:
        row['lastNetDebt'] = np.nan

    row['lastFreeCashFlow'] = cashflow.iloc[-1]['FreeCashFlow']
    row['avgFreeCashFlowGrowth'] = cashflow['avgFreeCashFlowGrowth'].mean()
    row['freeCashFlowStability'] = cashflow['avgFreeCashFlowGrowth'].max() - cashflow['avgFreeCashFlowGrowth'].min()

    row['modeGrade'] = grades.loc[(grades['epochGradeDate'].dt.month == dt.date.today().month)|
        (grades['epochGradeDate'].dt.month == dt.date.today().month-1),'toGrade'].mode()[0]
    row['lastGrade'] = grades.loc[grades['epochGradeDate'] == grades['epochGradeDate'].max(),'toGrade'][0]
    
    stocks_full_summary = pd.concat([stocks_full_summary, row], axis=0)

stocks_full_summary['debtToEBITDA'] = stocks_full_summary['lastNetDebt']/stocks_full_summary['ebitda']
stocks_full_summary.head(5)

MSFT


,address1,city,companyOfficers,country,fax,fullTimeEmployees,industry,longBusinessSummary,maxAge,phone,sector,state,website,zip,algorithm,ask,askSize,averageDailyVolume10Day,averageVolume,averageVolume10days,beta,bid,bidSize,coinMarketCapLink,currency,dayHigh,dayLow,dividendRate,dividendYield,exDividendDate,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiveYearAvgDividendYield,forwardPE,fromCurrency,lastMarket,marketCap,open,payoutRatio,previousClose,priceHint,priceToSalesTrailing12Months,regularMarketDayHigh,regularMarketDayLow,regularMarketOpen,regularMarketPreviousClose,regularMarketVolume,toCurrency,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,twoHundredDayAverage,volume,52WeekChange,SandP52WeekChange,bookValue,category,dateShortInterest,earningsQuarterlyGrowth,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,floatShares,forwardEps,fundFamily,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastSplitDate,lastSplitFactor,legalType,mostRecentQuarter,netIncomeToCommon,nextFiscalYearEnd,pegRatio,priceToBook,profitMargins,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortPercentOfFloat,shortRatio,trailingEps,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitda,ebitdaMargins,financialCurrency,freeCashflow,grossMargins,grossProfits,numberOfAnalystOpinions,operatingCashflow,operatingMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,strongBuy,buy,hold,sell,strongSell,longName,avgSurprice,lastSurprice,earning_dynamics,operatingMargin,grossMargin,operatingMarginStability,avgGrossMarginGrowth,avgOperatingMarginGrowth,avgNetIncomeGrowth,avgTotalRevenueGrowth,netIncomeStability,avgTotalAssetsGrowth,avgTotalDebtGrowth,avgNetDebtGrowth,netDebtStability,lastNetDebt,lastFreeCashFlow,avgFreeCashFlowGrowth,freeCashFlowStability,modeGrade,lastGrade,debtToEBITDA
MSFT,One Microsoft Way,Redmond,[],United States,425 706 7329,221000,Software—Infrastructure,"Microsoft Corporation develops, licenses, and ...",86400,425 882 8080,Technology,WA,https://www.microsoft.com,98052-6399,None,242.7,900,28734940,30824449,28734940,0.926403,242.55,1400,None,USD,245.165,239.65,2.72,0.0113,2023-02-15 01:00:00,241.5382,315.95,213.43,1.15,21.795149,None,None,1808305356800,241.1,0.267,240.22,2,8.904618,245.165,239.65,241.1,240.22,31933951,None,False,2.54,0.010574,26.111948,255.93135,31933951,-0.159139,-0.077274,23.276,None,2022-12-30 01:00:00,-0.144,17.813,8.67,1760605503488,7447689573,11.13,None,0.00059,0.73368,0,1668556800,0.68,2022-06-30 02:00:00,2003-02-18 01:00:00,2:1,None,2022-09-30 02:00:00,69788999680,2023-06-30 02:00:00,1.94,10.421894,0.34366,7454470144,0.0051,38191521,2022-11-30 01:00:00,40445360,0.0051,1.32,9.29,242.58,1.84,44.442,-0.133,98841001984,0.48672,USD,46155874304,0.6826,135620000000,48,87693000704,0.41691,1.585,buy,1.8,0.15223,0.42875,0.106,27.142,411.0,212.0,291.49,285.0,107244003328,14.387,77136003072,203074994176,14,13,6,0,1,Microsoft Corporation,0.02,0.022,1.055319,0.420553,0.684017,0.079183,0.008332,0.026394,0.233101,0.163778,0.25522,0.084102,-0.078465,-0.160989,0.066617,3.585000e+10,6.514900e+10,0.194608,0.079687,Buy,Outperform,0.362704


In [79]:
ticker.earning_history

maxAge  epsActual  epsEstimate  epsDifference  surprisePercent  \
symbol row                                                                   
MSFT   0         1       2.48         2.31           0.17            0.074   
       1         1       2.22         2.18           0.04            0.018   
       2         1       2.23         2.29          -0.06           -0.026   
       3         1       2.35         2.30           0.05            0.022   

               quarter period  
symbol row                     
MSFT   0    2021-12-31    -4q  
       1    2022-03-31    -3q  
       2    2022-06-30    -2q  
       3    2022-09-30    -1q

In [ ]:
stocks_full_summary.loc[stocks_full_summary['earningsGrowth'].isna()]

,address1,city,companyOfficers,country,fullTimeEmployees,industry,longBusinessSummary,maxAge,phone,sector,state,website,zip,algorithm,ask,askSize,averageDailyVolume10Day,averageVolume,averageVolume10days,beta,bid,bidSize,coinMarketCapLink,currency,dayHigh,dayLow,dividendRate,dividendYield,exDividendDate,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiveYearAvgDividendYield,forwardPE,fromCurrency,lastMarket,marketCap,open,payoutRatio,previousClose,priceHint,priceToSalesTrailing12Months,regularMarketDayHigh,regularMarketDayLow,regularMarketOpen,regularMarketPreviousClose,regularMarketVolume,toCurrency,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,twoHundredDayAverage,volume,52WeekChange,SandP52WeekChange,bookValue,category,dateShortInterest,earningsQuarterlyGrowth,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,floatShares,forwardEps,fundFamily,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastSplitDate,lastSplitFactor,legalType,mostRecentQuarter,netIncomeToCommon,nextFiscalYearEnd,pegRatio,priceToBook,profitMargins,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortPercentOfFloat,shortRatio,trailingEps,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitda,ebitdaMargins,financialCurrency,freeCashflow,grossMargins,grossProfits,numberOfAnalystOpinions,operatingCashflow,operatingMargins,quickRatio,recommendationKey,recommendationMean,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,strongBuy,buy,hold,sell,strongSell,longName,avgSurprice,lastSurprice,earning_dynamics,operatingMargin,grossMargin,operatingMarginStability,avgGrossMarginGrowth,avgOperatingMarginGrowth,avgNetIncomeGrowth,avgTotalRevenueGrowth,netIncomeStability,avgTotalAssetsGrowth,avgTotalDebtGrowth,avgNetDebtGrowth,netDebtStability,lastNetDebt,lastFreeCashFlow,avgFreeCashFlowGrowth,freeCashFlowStability,modeGrade,lastGrade,address2,fax,debtToEBITDA,TOTAL_SCORE
BRK-B,3555 Farnam Street,Omaha,[],United States,372000,Insurance—Diversified,"Berkshire Hathaway Inc., through its subsidiar...",86400,402 346 1400,Financial Services,NE,https://www.berkshirehathaway.com,68131,None,306.79,800,4361740,4174663,4361740,0.919899,306.56,900,None,USD,307.82,304.32,NaN,NaN,NaN,298.4184,362.1,259.85,NaN,20.094444,None,None,678343606272,304.38,0.0,302.0,2,2.293856,307.82,304.32,304.38,302.0,646682,None,False,0.0,0.0,59.01056,301.9704,646682,0.02578,-0.186294,310560.2,None,2022-11-30 01:00:00,NaN,1.638,0.055,16294445056,1205911,15.3,None,0.00441,0.65289,0,NaN,NaN,2021-12-31 01:00:00,2010-01-21 01:00:00,50:1,None,2022-09-30 02:00:00,-1336999936,2023-12-31 01:00:00,0.89,0.00099,-0.00452,1304380032,0.0024,5185763,2022-10-31 01:00:00,6398341,0.0045,1.32,5.21,307.445,1.459,25.12,NaN,9946999808,0.03364,USD,-23768750080,-0.00304,114863000000,1,34817998848,-0.00304,1.245,hold,2.7,-0.00062,-0.00102,0.09,200580.47,362.0,362.0,362.0,362.0,108957999104,74383.305,116495998976,295721992192,1,3,2,0,0,Berkshire Hathaway Inc.,0.2000,0.322,0.926346,0.314931,NaN,0.297179,NaN,0.099060,6.627330,0.188514,19.725686,0.107050,0.056210,-0.093199,1.379631,2.607800e+10,2.614500e+10,0.049562,0.201502,Equal-Weight,Equal-Weight,NaN,402 346 3375,2.621695,NaN
MO,6601 West Broad Street,Richmond,[],United States,6000,Tobacco,"Altria Group, Inc., through its subsidiaries, ...",86400,804 274 2200,Consumer Defensive,VA,https://www.altria.com,23230,None,45.59,1000,8591000,8564788,8591000,0.67852,45.58,800,None,USD,45.76,45.4009,3.76,0.0808,2022-12-21 01:00:00,45.5256,57.05,40.35,6.83,9.034862,None,None,81607720960,45.61,1.4163,45.5,2,3.944117,45.76,45.4009,45.61,45.5,1797544,None,False,3.64,0.08,17.718172,47.3383,1797544,-0.009386,-0.186294,-2.359,None,2022-11-30 01:00:00,NaN,8.631,5.183,107251556352,1790326680,5.04,None,0.00

In [ ]:
stocks_full_summary[['grossMargins','grossMargin']]

,grossMargins,grossMargin
AAPL,0.4331,0.433096
ABBV,0.69832,0.689556
ABR,0.91626,NaN
ABT,0.58163,0.569658
AFL,0.42594,NaN
AMD,0.5095,0.482475
AMGN,0.7578,0.751569
AMZN,0.43044,0.141149
ASML,0.51142,0.527054
ATVI,0.71129,0.736794


In [ ]:
# stocks_full_summary['TOTAL_SCORE'] = \
#     stocks_full_summary['marketCap']\
#     *stocks_full_summary['dividendYield']\
#     *stocks_full_summary['avgNetIncomeGrowth']\
#     *stocks_full_summary['avgFreeCashFlowGrowth']\
#     *stocks_full_summary['operatingMargins']*stocks_full_summary['operatingMargin']*stocks_full_summary['avgOperatingMarginGrowth']\
#     *stocks_full_summary['grossMargins']*stocks_full_summary['grossMargin']*stocks_full_summary['avgGrossMarginGrowth']\
#     *(stocks_full_summary['forwardEps'] / stocks_full_summary['trailingEps'])\
#     *stocks_full_summary['avgSurprice']\
#     *stocks_full_summary['totalCashPerShare']\
#     *stocks_full_summary['payoutRatio']
    # stocks_full_summary['freeCashflow'] * stocks_full_summary['lastFreeCashFlow'] * stocks_full_summary['avgFreeCashFlowGrowth']\
    # *stocks_full_summary['operatingCashflow']\
    # *stocks_full_summary['operatingMargins']*stocks_full_summary['operatingMargin']*stocks_full_summary['avgOperatingMarginGrowth']\
    # *stocks_full_summary['grossMargins']*stocks_full_summary['grossMargin']*stocks_full_summary['avgGrossMarginGrowth']\
    # *stocks_full_summary['dividendYield'] * stocks_full_summary['fiveYearAvgDividendYield'] * stocks_full_summary['trailingAnnualDividendYield']\
    # *(stocks_full_summary['forwardPE'] / stocks_full_summary['trailingPE'])\
    # *(stocks_full_summary['forwardEps'] / stocks_full_summary['trailingEps'])\
    # /(stocks_full_summary['avgNetDebtGrowth'] * stocks_full_summary['debtToEquity'] * stocks_full_summary['debtToEBITDA'])\
    # *(stocks_full_summary['returnOnAssets']*stocks_full_summary['returnOnEquity'])\
    # *(stocks_full_summary['revenueGrowth']*stocks_full_summary['revenuePerShare'])\
    # *(stocks_full_summary['ebitda']*stocks_full_summary['ebitdaMargins'])\
    # *(stocks_full_summary['avgNetIncomeGrowth'] * stocks_full_summary['avgTotalRevenueGrowth'])\
    # *(stocks_full_summary['avgTotalAssetsGrowth'] - stocks_full_summary['avgTotalDebtGrowth'])\
    # *stocks_full_summary['avgSurprice']\
    # *stocks_full_summary['totalCashPerShare']\
    # *stocks_full_summary['marketCap']\
    # *stocks_full_summary['payoutRatio']
stocks_full_summary[['longName','marketCap']].sort_values(by='marketCap', ascending=False)

,longName,marketCap
AAPL,Apple Inc.,2159683567616
MSFT,Microsoft Corporation,1817474367488
GOOGL,Alphabet Inc.,1162320281600
AMZN,"Amazon.com, Inc.",884334395392
BRK-B,Berkshire Hathaway Inc.,678343606272
JNJ,Johnson & Johnson,464148594688
XOM,Exxon Mobil Corporation,446443225088
TSLA,"Tesla, Inc.",438737797120
V,Visa Inc.,438432038912
NVDA,NVIDIA Corporation,411329527808


In [ ]:
isna_stocks = stocks_full_summary.isna().sum()
isna_stocks

address1              0
city                  0
companyOfficers       0
country               0
fullTimeEmployees     0
                     ..
lastGrade             0
address2             43
fax                  46
debtToEBITDA         11
TOTAL_SCORE          13
Length: 148, dtype: int64

In [ ]:
# cashflow = ticker.cash_flow().query('periodType=="12M"')
# cashflow.FreeCashFlow

In [ ]:
stocks_summary = stocks_full_summary[[
    'longName', 
    'sector',
    'industry',
    'marketCap',
    'currentPrice',
    'averageVolume',
    'earningsGrowth',
    'revenueGrowth',
    'trailingPE',
    'forwardPE',
    'priceToSalesTrailing12Months',
    'dividendYield',
    'fiveYearAvgDividendYield',
    'trailingAnnualDividendYield',
    'exDividendDate',
    'freeCashflow',
    'revenuePerShare',
    'targetHighPrice',
    'targetLowPrice',
    'targetMeanPrice',
    'targetMedianPrice',
    'recommendationKey',
    'modeGrade',
    'strongBuy','buy','hold','sell','strongSell'
    ]].copy()


stocks_summary['exDividendDate'] = pd.to_datetime(stocks_summary['exDividendDate']).dt.strftime('%Y-%m-%d')

stocks_summary['marketCap'] = stocks_summary['marketCap']/1e9
stocks_summary['freeCashflow'] = stocks_summary['freeCashflow']/1e9
stocks_summary['averageVolume'] = stocks_summary['averageVolume']/1e6
stocks_summary['targetHighPrice'] = stocks_summary['targetHighPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetLowPrice'] = stocks_summary['targetLowPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetMeanPrice'] = stocks_summary['targetMeanPrice']/stocks_summary['currentPrice']-1
stocks_summary['targetMedianPrice'] = stocks_summary['targetMedianPrice']/stocks_summary['currentPrice']-1
stocks_summary

,longName,sector,industry,marketCap,currentPrice,averageVolume,earningsGrowth,revenueGrowth,trailingPE,forwardPE,priceToSalesTrailing12Months,dividendYield,fiveYearAvgDividendYield,trailingAnnualDividendYield,exDividendDate,freeCashflow,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationKey,modeGrade,strongBuy,buy,hold,sell,strongSell
AAPL,Apple Inc.,Technology,Consumer Electronics,2159.683568,135.76,87.201515,0.048,0.081,22.21931,20.053175,5.476871,0.007,0.99,0.006803,2022-11-04,0.000002,24.317,0.576311,-0.101355,0.29972,0.296405,buy,Buy,11,21,6,0,0
ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers—General,286.396514,161.945,5.679307,0.242,0.033,21.592669,13.90086,4.953329,0.0367,4.35,0.035123,2023-01-12,0.0,32.638,0.234987,-0.166384,-0.006576,-0.01201,buy,Buy,4,6,10,0,0
ABR,"Arbor Realty Trust, Inc.",Real Estate,REIT—Mortgage,2.343018,13.66,2.206565,-0.285,-0.054,7.189474,9.046357,3.608657,0.119,9.12,0.114925,2022-11-17,0.0,4.091,0.354319,0.061493,0.207906,0.24451,buy,Market Outperform,0,2,1,0,0
ABT,Abbott Laboratories,Healthcare,Medical Devices,188.20096,107.94,5.985533,-0.308,-0.047,24.36569,24.476192,4.179457,0.0191,1.53,0.017654,2023-01-12,0.0,25.592,0.334074,-0.166203,0.082824,0.088568,buy,Outperform,7,6,7,0,0
AFL,Aflac Incorporated,Financial Services,Insurance—Life,44.669317,71.84,2.715812,0.917,-0.08,9.1867,13.157509,2.134428,0.0244,2.35,0.021653,2023-02-14,0.0,32.465,0.113586,-0.24833,-0.056654,-0.039532,hold,Equal-Weight,1,2,10,1,1
AMD,"Advanced Micro Devices, Inc.",Technology,Semiconductors,108.592439,67.34,80.917463,-0.947,0.29,41.8323,18.452053,4.756985,NaN,NaN,0.0,1995-04-27,0.0,15.646,1.970003,-0.108999,0.349421,0.299376,buy,Buy,5,6,15,4,1
AMGN,Amgen Inc.,Healthcare,Drug Manufacturers—General,142.153449,266.415,2.890631,0.202,-0.008,21.381622,14.424202,5.398916,0.0322,2.82,0.028631,2023-02-14,0.0,48.29,0.219901,-0.324362,0.00092,0.015333,hold,Overweight,3,7,15,0,0
AMZN,"Amazon.com, Inc.",Consumer Cyclical,Internet Retail,884.334395,86.685,79.831473,-0.097,0.147,79.52752,52.21988,1.760952,NaN,NaN,0.0,NaN,0.000001,49.349,1.685009,-0.077118,0.613774,0.580435,buy,Buy,15,28,3,1,0
ASML,ASML Holding N.V.,Technology,Semiconductor Equipment & Materials,238.440071,576.9219,1.402849,0.007,0.102,38.90235,29.13747,12.085888,0.0116,0.83,0.009003,2022-11-03,0.0,49.224,0.565567,-0.181865,0.196678,0.148232,buy,Overweight,2,3,2,0,0
ATVI,"Activision Blizzard, Inc.",Communication Services,Electronic Gaming & Multimedia,59.193844,75.635,6.865368,-0.329,-0.139,35.509388,19.443445,8.045921,0.0062,0.59,0.006194,2022-04-13,0.0,9.432,0.322139,0.031269,0.220202,0.256032,buy,Buy,9,11,7,0,0


In [ ]:
# yq.Ticker('AAPL').earnings
# pd.DataFrame(yq.Ticker('AAPL').all_modules)

funds

In [ ]:
yq.Ticker('SCHD').fund_performance

{'SCHD': {'maxAge': 1,
  'performanceOverview': {'asOfDate': '2022-12-14 01:00:00',
   'ytdReturnPct': -0.0095,
   'oneYearTotalReturn': 0.026700001,
   'threeYearTotalReturn': 0.1433},
  'performanceOverviewCat': {'ytdReturnPct': 0.109,
   'fiveYrAvgReturnPct': 0.15359999},
  'trailingReturns': {'asOfDate': '2022-12-12 01:00:00',
   'ytd': -0.0138792,
   'oneMonth': 0.0096464,
   'threeMonth': 0.051581603,
   'oneYear': 0.0182523,
   'threeYear': 0.1406709,
   'fiveYear': 0.13403301,
   'tenYear': 0.13838139,
   'lastBullMkt': 0.0,
   'lastBearMkt': 0.0},
  'trailingReturnsNav': {'ytd': -0.0138792,
   'oneMonth': 0.0096464,
   'threeMonth': 0.051581603,
   'oneYear': 0.0182523,
   'threeYear': 0.1406709,
   'fiveYear': 0.13403301,
   'tenYear': 0.13838139},
  'trailingReturnsCat': {'ytd': 0.109,
   'oneMonth': -0.001,
   'threeMonth': 0.0326,
   'oneYear': 0.17040001,
   'threeYear': 0.105299994,
   'fiveYear': 0.15359999,
   'tenYear': 0.0655,
   'lastBullMkt': 0.0,
   'lastBearMkt':

In [ ]:
yq.Ticker('SCHD').fund_profile

{'SCHD': {'maxAge': 1,
  'styleBoxUrl': 'https://s.yimg.com/lq/i/fi/3_0stylelargeeq1.gif',
  'family': 'Schwab ETFs',
  'categoryName': 'Large Value',
  'legalType': 'Exchange Traded Fund',
  'managementInfo': {'managerName': None, 'managerBio': None},
  'feesExpensesInvestment': {'annualReportExpenseRatio': 0.00059999997,
   'annualHoldingsTurnover': 0.14,
   'totalNetAssets': 75870.54,
   'projectionValues': {}},
  'feesExpensesInvestmentCat': {'annualReportExpenseRatio': 0.0034,
   'annualHoldingsTurnover': 34.69,
   'totalNetAssets': 75870.54,
   'projectionValuesCat': {}},
  'brokerages': []}}

In [ ]:
yq.Ticker('SCHD').fund_sector_weightings

,SCHD
0,
realestate,0.0000
consumer_cyclical,0.0747
basic_materials,0.0196
consumer_defensive,0.1339
technology,0.1633
communication_services,0.0487
financial_services,0.2036
utilities,0.0037
industrials,0.1640


In [ ]:
yq.Ticker('SCHD').fund_top_holdings

symbol                           holdingName  holdingPercent
symbol row                                                             
SCHD   0      MRK                        Merck & Co Inc          0.0457
       1     AMGN                             Amgen Inc          0.0433
       2      IBM  International Business Machines Corp          0.0425
       3     AVGO                          Broadcom Inc          0.0415
       4      PEP                           PepsiCo Inc          0.0411
       5      BLK                         BlackRock Inc          0.0405
       6      LMT                  Lockheed Martin Corp          0.0405
       7       VZ            Verizon Communications Inc          0.0396
       8     CSCO                     Cisco Systems Inc          0.0396
       9      PFE                            Pfizer Inc          0.0396

In [ ]:
yq.Ticker('SPHD').fund_top_holdings

symbol                      holdingName  holdingPercent
symbol row                                                        
SPHD   0      IRM                Iron Mountain Inc          0.0321
       1       MO                 Altria Group Inc          0.0314
       2      PPL                         PPL Corp          0.0308
       3      WMB           Williams Companies Inc          0.0291
       4        T                         AT&T Inc          0.0289
       5      KMI        Kinder Morgan Inc Class P          0.0272
       6       PM  Philip Morris International Inc          0.0253
       7      CVX                     Chevron Corp          0.0244
       8      PRU         Prudential Financial Inc          0.0234
       9     AMCR        Amcor PLC Ordinary Shares          0.0232

In [ ]:
yq.Ticker('SCHD').price

{'SCHD': {'maxAge': 1,
  'preMarketChangePercent': -0.0105154,
  'preMarketChange': -0.809998,
  'preMarketTime': '2022-12-15 15:29:58',
  'preMarketPrice': 76.22,
  'preMarketSource': 'FREE_REALTIME',
  'postMarketChangePercent': -0.00039754115,
  'postMarketChange': -0.030006409,
  'postMarketTime': 1671143702,
  'postMarketPrice': 75.45,
  'postMarketSource': 'FREE_REALTIME',
  'regularMarketChangePercent': -0.02012197,
  'regularMarketChange': -1.5499954,
  'regularMarketTime': '2022-12-15 22:00:00',
  'priceHint': 2,
  'regularMarketPrice': 75.48,
  'regularMarketDayHigh': 76.4299,
  'regularMarketDayLow': 75.1401,
  'regularMarketVolume': 3815032,
  'regularMarketPreviousClose': 77.03,
  'regularMarketSource': 'DELAYED',
  'regularMarketOpen': 76.34,
  'exchange': 'PCX',
  'exchangeName': 'NYSEArca',
  'exchangeDataDelayedBy': 0,
  'marketState': 'POST',
  'quoteType': 'ETF',
  'symbol': 'SCHD',
  'underlyingSymbol': None,
  'shortName': 'Schwab US Dividend Equity ETF',
  'longNa

In [ ]:
yq.Ticker('SCHD').key_stats

{'SCHD': {'maxAge': 1,
  'priceHint': 2,
  'category': 'Large Value',
  'ytdReturn': -0.0095,
  'beta3Year': 0.89,
  'totalAssets': 44960911360,
  'yield': 0.031400003,
  'fundFamily': 'Schwab ETFs',
  'fundInceptionDate': '2011-10-20 02:00:00',
  'legalType': 'Exchange Traded Fund',
  'threeYearAverageReturn': 0.1433,
  'fiveYearAverageReturn': 0.1245,
  'lastSplitFactor': None}}

In [ ]:
pd.DataFrame(yq.Ticker('SCHD').all_modules)

,SCHD
assetProfile,"{'phone': 'NA', 'longBusinessSummary': 'To pur..."
defaultKeyStatistics,"{'maxAge': 1, 'priceHint': 2, 'category': 'Lar..."
fundPerformance,"{'maxAge': 1, 'performanceOverview': {'asOfDat..."
fundProfile,"{'maxAge': 1, 'styleBoxUrl': 'https://s.yimg.c..."
pageViews,"{'shortTermTrend': 'DOWN', 'midTermTrend': 'UP..."
price,"{'maxAge': 1, 'preMarketSource': 'FREE_REALTIM..."
quoteType,"{'exchange': 'PCX', 'quoteType': 'ETF', 'symbo..."
summaryDetail,"{'maxAge': 1, 'priceHint': 2, 'previousClose':..."
summaryProfile,"{'phone': 'NA', 'longBusinessSummary': 'To pur..."
topHoldings,"{'maxAge': 1, 'stockPosition': 0.9995, 'bondPo..."


screener

In [ ]:
s = yq.Screener()
s.available_screeners
# s.get_screeners('all_cryptocurrencies_us')

['accident_health_insurance',
 'advertising_agencies',
 'aerospace_defense_major_diversified',
 'aerospace_defense_products_services',
 'aggressive_small_caps',
 'agricultural_chemicals',
 'air_delivery_freight_services',
 'air_services_other',
 'all_cryptocurrencies_au',
 'all_cryptocurrencies_ca',
 'all_cryptocurrencies_eu',
 'all_cryptocurrencies_gb',
 'all_cryptocurrencies_in',
 'all_cryptocurrencies_us',
 'aluminum',
 'apparel_stores',
 'appliances',
 'application_software',
 'asset_management',
 'auto_dealerships',
 'auto_manufacturers_major',
 'auto_parts',
 'auto_parts_stores',
 'auto_parts_wholesale',
 'basic_materials',
 'basic_materials_wholesale',
 'beverages_brewers',
 'beverages_soft_drinks',
 'beverages_wineries_distillers',
 'biotechnology',
 'broadcasting_radio',
 'broadcasting_tv',
 'building_materials_wholesale',
 'business_equipment',
 'business_services',
 'business_software_services',
 'catalog_mail_order_houses',
 'catv_systems',
 'cement',
 'chemicals_major_dive